In [1]:
# Import packages

import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyart
import glob
import matplotlib.patheffects as path_effects
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from pyproj import Transformer
import radlib
import os
import h5py
import matplotlib.dates as mdates
from scipy.ndimage import gaussian_filter1d
from cartopy.feature import NaturalEarthFeature
import xmltodict, geopandas, geojson, xml #xml and json do not exist
from datetime import datetime, timedelta
import geopy.distance
from datetime import datetime, timedelta
import numpy.matlib as npm
import copy
from scipy.signal import convolve2d
from astropy.convolution import convolve
import scipy.ndimage as ndi
import re
from skimage.draw import polygon

from pprint import pprint
from pysteps import io, motion, rcparams
from pysteps.utils import conversion, transformation
from pysteps.visualization import plot_precip_field, quiver


os.environ["library_metranet_path"] = "/store_new/mch/msrad/idl/lib/radlib4/" # needed for pyradlib
os.environ["METRANETLIB_PATH"] = "/store_new/mch/msrad/idl/lib/radlib4/" # needed for pyart_mch




## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

Pysteps configuration file found at: /scratch/mch/fackerma/miniforge3/envs/testenv/lib/python3.12/site-packages/pysteps/pystepsrc



## Definitions

In [2]:
# Defining valid time and elevation scan

valid_time3 = '0655'
elevation_scan = '5'
radar_station = 'W'
radar_name = 'Weissfluhgipfel'

In [3]:
# Data Imports, processing, norming, etc.

# GPSConverter for coordinate transformation
class GPSConverter(object):
    '''
    GPS Converter class which is able to perform convertions between the 
    CH1903 and WGS84 system.
    '''
    # Convert CH y/x/h to WGS height
    def CHtoWGSheight(self, y, x, h):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        h = (h + 49.55) - (12.60 * y_aux) - (22.64 * x_aux)
        return h

    # Convert CH y/x to WGS lat
    def CHtoWGSlat(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lat = (16.9023892 + (3.238272 * x_aux)) + \
                - (0.270978 * pow(y_aux, 2)) + \
                - (0.002528 * pow(x_aux, 2)) + \
                - (0.0447 * pow(y_aux, 2) * x_aux) + \
                - (0.0140 * pow(x_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lat = (lat * 100) / 36
        return lat

    # Convert CH y/x to WGS long
    def CHtoWGSlng(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lng = (2.6779094 + (4.728982 * y_aux) + \
                + (0.791484 * y_aux * x_aux) + \
                + (0.1306 * y_aux * pow(x_aux, 2))) + \
                - (0.0436 * pow(y_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lng = (lng * 100) / 36
        return lng

    # Convert decimal angle (° dec) to sexagesimal angle (dd.mmss,ss)
    def DecToSexAngle(self, dec):
        degree = int(math.floor(dec))
        minute = int(math.floor((dec - degree) * 60))
        second = (((dec - degree) * 60) - minute) * 60
        return degree + (float(minute) / 100) + (second / 10000)
		
    # Convert sexagesimal angle (dd.mmss,ss) to seconds
    def SexAngleToSeconds(self, dms):
        degree = 0 
        minute = 0 
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return second + (minute * 60) + (degree * 3600)

    # Convert sexagesimal angle (dd.mmss) to decimal angle (degrees)
    def SexToDecAngle(self, dms):
        degree = 0
        minute = 0
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return degree + (minute / 60) + (second / 3600)
    
    # Convert WGS lat/long (° dec) and height to CH h
    def WGStoCHh(self, lat, lng, h):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        h = (h - 49.55) + (2.73 * lng_aux) + (6.94 * lat_aux)
        return h

    # Convert WGS lat/long (° dec) to CH x
    def WGStoCHx(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        x = ((200147.07 + (308807.95 * lat_aux) + \
            + (3745.25 * pow(lng_aux, 2)) + \
            + (76.63 * pow(lat_aux,2))) + \
            - (194.56 * pow(lng_aux, 2) * lat_aux)) + \
            + (119.79 * pow(lat_aux, 3))
        return x

	# Convert WGS lat/long (° dec) to CH y
    def WGStoCHy(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        y = (600072.37 + (211455.93 * lng_aux)) + \
            - (10938.51 * lng_aux * lat_aux) + \
            - (0.36 * lng_aux * pow(lat_aux, 2)) + \
            - (44.54 * pow(lng_aux, 3))
        return y

    def LV03toWGS84(self, east, north, height):
        '''
        Convert LV03 to WGS84 Return a array of double that contain lat, long,
        and height
        '''
        d = []
        d.append(self.CHtoWGSlat(east, north))
        d.append(self.CHtoWGSlng(east, north))
        d.append(self.CHtoWGSheight(east, north, height))
        return d
        
    def WGS84toLV03(self, latitude, longitude, ellHeight):
        '''
        Convert WGS84 to LV03 Return an array of double that contaign east,
        north, and height
        '''
        d = []
        d.append(self.WGStoCHy(latitude, longitude))
        d.append(self.WGStoCHx(latitude, longitude))
        d.append(self.WGStoCHh(latitude, longitude, ellHeight))
        return d
        
if __name__ == "__main__":
    ''' Example usage for the GPSConverter class.'''

    converter = GPSConverter()

    # Coordinates
    wgs84 = []
    lv03  = [567719, 155077, 1974]
    #567719/ 155077  Altitude: 1974
    # Convert WGS84 to LV03 coordinates
    wgs84 = converter.LV03toWGS84(lv03[0], lv03[1], lv03[2])

    print("WGS84: ")
    print(wgs84)
    print("LV03: ")
    print(lv03)

# Transform function for dealiased velocity
def transform_from_digital(mydata, nyquist):
    myfinaldata = (mydata * 1.0 - 128) * nyquist / 127
    myfinaldata[mydata == 0] = np.nan
    return myfinaldata

# Convert polar coordinates (range, azimuth) to geographic coordinates (lat/lon)
def radar_to_latlon(radar_lat, radar_lon, ranges, azimuths):
    R = 6371.0  # Earth radius in kilometers
    azimuths_rad = np.radians(azimuths)
    latitudes = np.zeros((len(azimuths), len(ranges)))
    longitudes = np.zeros((len(azimuths), len(ranges)))

    for i, az in enumerate(azimuths_rad):
        latitudes[i, :] = radar_lat + (ranges / R) * np.cos(az) * (180.0 / np.pi)
        longitudes[i, :] = radar_lon + (ranges / R) * np.sin(az) * (180.0 / np.pi) / np.cos(np.radians(radar_lat))

    return latitudes, longitudes

# Define TRT reading function
def read_TRT(path, file=0, ttime=0):
    """
    Read .trt or .json file containing TRT output
    Returns dataframe with attributes and gridded TRT cells

    Parameters
    ----------

    path : string
        path, where to look for files.
    file: string
        filename
    ttime : string
        timestep to find files for.
    Requires either filename or timestep
   
    Returns
    -------
    trt_df : dataframe
        TRT cells and attributes of the timestep.
    cells: list
        Gridded TRT cells per timestep
    timelist: list
        timesteps

    """
   
    o_x=255000
    o_y=-160000
    lx=710; ly=640
    cells=np.zeros([ly,lx])
    if file == 0:
        file=glob.glob(path["lomdata"]+'TRTC/*'+ttime+'*json*')
        if len(file)>0: flag=1
        else:
            file=glob.glob(path["lomdata"]+'TRTC/*'+ttime+'*'+'.trt')[0]
            flag=0
    else:
        if 'json' in file: flag=1; ttime=file[-20:-11]
        else: flag=0; ttime=file[-15:-6]
        file=[file]
   
    if flag==1:
        with open(file[0]) as f: gj = geojson.FeatureCollection(gs.load(f))
        trt_df=geopandas.GeoDataFrame.from_features(gj['features'])
        if len(trt_df)>0:
          # print(trt_df.lon.values.astype(float))
          chx, chy = c_transform(trt_df.lon.values.astype(float),trt_df.lat.values.astype(float))
          trt_df['chx']=chx.astype(str); trt_df['chy']=chy.astype(str)
          for n in range(len(trt_df)):
              lon,lat=trt_df.iloc[n].geometry.boundary.xy
              # print(trt_df.iloc[n])
              chx, chy = c_transform(lon,lat)
              # trt_df.iloc[n]['chx']=chx.astype(str); trt_df.iloc[n]['chy']=chy.astype(str)
              #transform.c_transform(trt_df.iloc[n].lon.values,trt_df.iloc[n].lat.values)
              ix=np.round((chx-o_x)/1000).astype(int)
              iy=np.round((chy-o_y)/1000).astype(int)
              rr, cc = polygon(iy, ix, cells.shape)
              # print(lat,lon,chx,chy,ix,iy)
              cells[rr,cc]=int(trt_df.traj_ID.iloc[n]);
        else: cells=[]
    else:
        data=pd.read_csv(file).iloc[8:]
        headers=pd.read_csv(file).iloc[7:8].iloc[0][0].split()
        trt_df=pd.DataFrame()
        for n in range(len(data)):
            t=data.iloc[n].str.split(';',expand=True)
            trt_df.loc[n,'traj_ID']=int(t[0].values)
            trt_df.loc[n,'time']=int(t[1].values)
            trt_df.loc[n,'lon']=t[2].values.astype(float)
            trt_df.loc[n,'lat']=t[3].values.astype(float)
            chx,chy=c_transform([trt_df.loc[n,'lon']],[trt_df.loc[n,'lat']])
            ix=np.round((chx-o_x)/1000).astype(int)
            if ix>=710: ix=709
            iy=np.round((chy-o_y)/1000).astype(int)
            if iy>=640: iy=639
            n2=27
            if int(ttime)>=221520631: n2=82
            tt=np.array(t)[0,n2:-1]
            tt=np.reshape(tt,[int(len(tt)/2),2])
            trt_df.loc[n,'chx']=chx
            trt_df.loc[n,'chy']=chy
            lat=tt[:,1].astype(float); lon=tt[:,0].astype(float)
            # trt_df=trt_df.astype(str)
            chx,chy=c_transform(lon,lat)
            ix=np.round((chx-o_x)/1000).astype(int)
            iy=np.round((chy-o_y)/1000).astype(int)
            rr, cc = polygon(iy, ix, cells.shape)
            cells[rr,cc]=int(t[0].values);
    # print(np.nanmax(cells))
    timelist=[str(ttime)]
    return trt_df, [cells], timelist

# Define VAD reading function
def read_VAD(file_VAD):
    mydoc = xml.dom.minidom.parse(file_VAD)

    levels = mydoc.getElementsByTagName('level')
   
    heights = mydoc.getElementsByTagName('height')
    speeds = mydoc.getElementsByTagName('speed')
    directions = mydoc.getElementsByTagName('direction')
   
    vad_levels=np.zeros(len(levels))
    vad_heights=np.zeros(len(levels))
    vad_speeds=np.zeros(len(levels))
    vad_directions=np.zeros(len(levels))
    for m in range(0,len(levels)):
        vad_levels[m]=levels[m].firstChild.data
        vad_heights[m]=heights[m].firstChild.data
        vad_speeds[m]=speeds[m].firstChild.data
        vad_directions[m]=directions[m].firstChild.data
    vad_s=np.zeros(150); vad_d=np.zeros(150); vad_s[:]=np.nan; vad_d[:]=np.nan;
    for n in range(0,150):
        for m in range(0,len(levels)):
            if vad_levels[m]==n:
                vad_s[n]=vad_speeds[m]
                vad_d[n]=vad_directions[m]
       
       
    vad_u = np.multiply(vad_s, np.cos(np.radians(vad_d)))
    vad_v = np.multiply(vad_s, np.sin(np.radians(vad_d)))
   
    vadu=np.zeros(15); vadv=np.zeros(15);
    for n in range(0,15):
        a=n*10
        vadu[n]=np.nanmean(vad_u[a:a+10])
        vadv[n]=np.nanmean(vad_v[a:a+10])
       
    return vadu, vad_u, vadv, vad_v

# Function to calculate azimuth angle from radar to downburst location
def calculate_azimuth(lat1, lon1, lat2, lon2):
    # Convert to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    # Calculate azimuth
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    azimuth = math.atan2(x, y)
    azimuth_deg = math.degrees(azimuth)
    
    # Normalize to 0-360 degrees
    return (azimuth_deg + 360) % 360

# Convolves and normalizes 2D data with a 3x3 kernel
def conv(data):
    """
    Convolves and normalizes 2D data with a 3x3 kernel

    Parameters
    ----------
    data : 2-D array
        data to be convolved.

    Returns
    -------
    d_conv : 2-D array
        convolved data.

    """
    ## convolves 2D data with a 3x3 kernel, equal weights, wrapped boundary, NaN values ignored / filled in
    g_kernel=np.ones([3,3])/9
    d_conv=convolve(data,g_kernel, boundary='wrap')
    return d_conv

# azimuthal centered difference derivative for polar data -> wraps around 0/360
def az_cd(myfinaldata, nyquist, threshold, resolution, min_size):
    """
    azimuthal centered difference derivative for polar data -> wraps around 0/360

    Parameters
    ----------
    myfinaldata : 2D array
        input data.
    nyquist : float
        Nyquist velocity of raw velocity data.
    threshold : float
        shear correction threshold.
    resolution : float
        radial resolution of data.
    min_size : int
        minimum number of gates required for correction.

    Returns
    -------
    myshear_cor : 2D array
        azimuthal derivative, corrected for unfolding errors.
    mygateshear_cor : 2D array
        azimuthal difference (no normalization by distance), corrected for unfolding errors.

    """
    ## azimuthal derivative, centered difference
    ## corrects for anomalous shear exceeding threshold
    ## shear only corrected if contiguous area of several pixels
    myfinaldata_1=np.zeros(myfinaldata.shape)
    myfinaldata_1[:-1,:]=myfinaldata[1:,:]
    myfinaldata_1[-1,:]=myfinaldata[0,:]
    myfinaldata_2=np.zeros(myfinaldata.shape)
    myfinaldata_2[1:,:]=myfinaldata[:-1,:]
    myfinaldata_2[0,:]=myfinaldata[-1,:]
    
    distance=np.arange(0.5*resolution, myfinaldata.shape[1]*resolution 
                       + 0.5*resolution, resolution)
    distance=npm.repmat(distance,myfinaldata.shape[0],1)
    distance=np.divide(np.multiply(distance,2*np.pi),360)
    
    myshear_1=(myfinaldata-myfinaldata_1)/(2*distance)*-(1)
    myshear_2=(myfinaldata_2-myfinaldata)/(2*distance)*(-1)
    myshear_3=(myfinaldata_2-myfinaldata_1)/(2*distance)*(-1)
    
    myshear_1_cor=shear_cor(myshear_1, distance, threshold, nyquist, min_size)
        
    myshear_2_cor=shear_cor(myshear_2, distance, threshold, nyquist, min_size)

    myshear_3_cor=shear_cor(myshear_3, distance, threshold, nyquist, min_size)
    
    myshear_cor=np.nansum([myshear_2_cor,myshear_1_cor],axis=0)
    myshear_cor[np.isnan(myfinaldata)==1]=myshear_3_cor[np.isnan(myfinaldata)==1]
    mygateshear_cor=myshear_cor*distance
    
    return myshear_cor, mygateshear_cor

# Identifies unfolding errors in derivative and corrects for them
def shear_cor(myshear, distance, threshold, nyquist, min_size):
    """
    Identifies unfolding errors in derivative and corrects for them

    Parameters
    ----------
    myshear : 2D array
        input data.
    distance : 2D array
        azimuthal Cartesian distance between gates.
    threshold : float
        error identification threshold.
    nyquist : float
        correction interval (nyquist velocity).
    min_size : int
        minimum number of connected erroneous gates.

    Returns
    -------
    myshear_cor : 2D array
        corrected data.

    """
    ## function embedded in centered difference shear
    ## eliminates anomalous shear in contiguous area
    myshear_cor=(myshear*(2*distance))
    thresh_1=(myshear*(2*distance))-threshold
    thresh_2=(myshear*(2*distance))+threshold
    
    mybin_pos=np.zeros(myshear.shape)
    mybin_pos[thresh_1>=0]=1
    labels, n_groups=ndi.label(mybin_pos)
    for n in range(1, n_groups):
        size=len(np.where(labels==n)[0])
        if size<min_size: labels[labels==n]=0
        else: myshear_cor[labels==n] -= (2*nyquist)
    
    mybin_neg=np.zeros(myshear.shape)
    mybin_neg[thresh_2<=0]=1
    labels, n_groups=ndi.label(mybin_neg)
    for n in range(1, n_groups):
        size=len(np.where(labels==n)[0])
        if size<min_size: labels[labels==n]=0
        else: myshear_cor[labels==n] += (2*nyquist)
    myshear_cor=myshear_cor/(2*distance)
    return myshear_cor

# azimuthal linear least squares derivative        
def az_llsd(d_r, d_theta, u_k_r, u_k_thet, u_k, weights):
    """
    azimuthal linear least squares derivative

    Parameters
    ----------
    d_r : array of same dimensions as data
        radial distance.
    d_theta : array of same dimensions as data
        azimuthal distance.
    u_k_r : array of same dimensions as data
        data weighted radially.
    u_k_thet : array of same dimensions as data
        data weighted azimuthally.
    u_k : array of same dimensions as data
        data weighted uniformly.
    weights : array of same dimensions as data
        weights for convolution.

    Returns
    -------
    az_shear : array of same dimensions as data
        azimuthal derivative of data.

    """
    ##azimuthal linear least squares derivative
    az_shear=( np.sum(weights * d_r * d_theta) * np.sum(weights * d_r * d_theta) * np.sum(weights) \
                + np.sum(weights * d_theta * d_theta) * np.sum(weights * d_r) * np.sum(weights * d_r) \
                + np.sum(weights * d_theta) * np.sum(weights * d_r * d_r) * np.sum(weights * d_theta) \
                - 2 * np.sum(weights * d_r * d_theta) * np.sum(weights * d_r) * np.sum(weights * d_theta) \
                - np.sum(weights * d_theta * d_theta) * np.sum(weights * d_r * d_r) * np.sum(weights)) ** (-1) \
                * ( u_k_r * (np.sum(weights * d_r * d_theta) * np.sum(weights) - np.sum(weights * d_theta) * np.sum(weights * d_r)) \
                + u_k_thet * ( - np.sum(weights * d_r * d_r) * np.sum(weights) + np.sum(weights * d_r) * np.sum(weights * d_r)) \
                + u_k * (np.sum(weights * d_r * d_r) * np.sum(weights * d_theta) - np.sum(weights * d_r) * np.sum(weights * d_r * d_theta)) )
    return az_shear

# radial linear least squares derivative
def div_llsd(d_r, d_theta, u_k_r, u_k_thet, u_k, weights):
    """
    radial linear least squares derivative

    Parameters
    ----------
    d_r : array of same dimensions as data
        radial distance.
    d_theta : array of same dimensions as data
        azimuthal distance.
    u_k_r : array of same dimensions as data
        data weighted radially.
    u_k_thet : array of same dimensions as data
        data weighted azimuthally.
    u_k : array of same dimensions as data
        data weighted uniformly.
    weights : array of same dimensions as data
        weights for convolution.

    Returns
    -------
    div_shear : array of same dimensions as data
        radial derivative of data.

    """
    ##radial linear least squares derivative
    div_shear= ( np.sum(weights * d_r * d_theta) * np.sum(weights * d_r * d_theta) * np.sum(weights) \
                + np.sum(weights * d_theta * d_theta) * np.sum(weights * d_r) * np.sum(weights * d_r) \
                + np.sum(weights * d_theta) * np.sum(weights * d_r * d_r) * np.sum(weights * d_theta) \
                - 2 * np.sum(weights * d_r * d_theta) * np.sum(weights * d_r) * np.sum(weights * d_theta) \
                - np.sum(weights * d_theta * d_theta) * np.sum(weights * d_r * d_r) * np.sum(weights)) ** (-1) \
                * ( u_k_r * (- np.sum(weights * d_theta * d_theta) * np.sum(weights) + np.sum(weights * d_theta) * np.sum(weights * d_theta)) \
                + u_k_thet * (np.sum(weights * d_r * d_theta) * np.sum(weights) - np.sum(weights * d_theta) * np.sum(weights * d_r)) \
                + u_k * ( - np.sum(weights * d_r * d_theta) * np.sum(weights * d_theta) + np.sum(weights * d_r) * np.sum(weights * d_theta * d_theta)) )
    return div_shear

# full linear least squares derivative
def llsd(vel, az_min, az_max, w_k, r_k, resolution):
    """
    full linear least squares derivative

    Parameters
    ----------
    vel : 2D array
        input data, polar data.
    az_min : int
        min. number of gates in azimuth.
    az_max : int
        max. number of gates in azimuth.
    w_k : float
        kernel width in azimuth (Cartesian).
    r_k : float
        kernel width in radial (Cartesian).
    resolution : float
        radial resolution.

    Returns
    -------
    az_shear : 2D array
        azimuthal derivative.
    div_shear : 2D array
        radial derivative.

    """
    ##complete linear least squares derivative
    az_shear=np.zeros(vel.shape); az_shear[:]=np.nan
    div_shear=np.zeros(vel.shape); div_shear[:]=np.nan
    
    distance=np.arange(0.5*resolution, vel.shape[1]*resolution 
                       + 0.5*resolution, resolution)
    distance=npm.repmat(distance,vel.shape[0],1)
    distance=np.divide(np.multiply(distance,2*np.pi),360)

    r_h=int(np.floor(az_max/2))
	
    for n2 in range(r_h,vel.shape[1]-r_h):
        
        az_w=int(np.round(w_k/distance[1,n2]));
        if az_w%2 == 0: az_w+=1
        if az_w>az_max: az_w=az_max
        if az_w<az_min: az_w=az_min
        az_r=int(np.floor(az_w/2))
        
        r_w= r_k/resolution;
        if r_w%2 == 0: r_w+=1
        r_w=int(r_w)
        r_r=r_w*resolution
        r_vec=np.arange(-r_r,r_r+resolution,resolution)
        r_vec=np.expand_dims(r_vec,axis=0)
        d_r=np.repeat(r_vec,az_w,axis=0)
        
        d_r=np.zeros([az_w,r_w])
        for n2_2 in range(0,r_w):
            d_r[:,n2_2]=-r_k*n2_2/(r_w-1)+r_k*0.5
        rw_f=int(np.floor(r_w/2))
	    
        d_r2=-copy.deepcopy(d_r); #print(d_r.shape,n2)
        weights=np.ones([az_w,r_w])
        d_theta=copy.deepcopy(distance[:az_w,n2-rw_f:n2+rw_f+1]); d_theta[az_r,:]=0
        d_theta[:az_r,:]=-d_theta[:az_r,:];
        d_thet=copy.deepcopy(d_theta); d_thet=-np.flip(d_thet,1)

        u_k=convolve2d(vel[:,n2-rw_f:n2+rw_f+1],weights,mode='same',boundary='wrap')[:,1]
        u_k_thet=convolve2d(vel[:,n2-rw_f:n2+rw_f+1],d_thet,mode='same',boundary='wrap')[:,1]
        u_k_r=convolve2d(vel[:,n2-rw_f:n2+rw_f+1],d_r2,mode='same',boundary='wrap')[:,1]

        az_shear[:,n2]= az_llsd(d_r, d_theta, u_k_r, u_k_thet, u_k, weights)
        div_shear[:,n2]= div_llsd(d_r, d_theta, u_k_r, u_k_thet, u_k, weights)
    return az_shear, div_shear

# c_transform
def c_transform(lon,lat):
    """
    transforms arrays of lat/lon to chx/chy

    Parameters
    ----------
    lon : float
        longitude.
    lat : float
        latitude.

    Returns
    -------
    chx : float
        chx in m.
    chy : float
        chy in m.

    """
    converter = GPSConverter()
    chx=np.zeros([len(lon)])
    chy=np.zeros([len(lon)])
    for n in range(len(lon)):
        chx[n],chy[n],z=converter.WGS84toLV03(lat[n], lon[n], 0)
    return chx,chy

# transform_c 
def transform_c(chx,chy):
    """
    transforms arrays of chx/chy to lat/lon
    
    Parameters
    -------
    chx : float
        chx in m.
    chy : float
        chy in m.
        
    Returns
    ----------
    lon : float
        longitude.
    lat : float
        latitude.



    """
    converter = GPSConverter()
    lon=np.zeros([len(chx)])
    lat=np.zeros([len(chy)])
    for n in range(len(lon)):
        lat[n], lon[n],z=converter.LV03toWGS84(chx[n],chy[n], 0)
    return lon,lat



# Radar locations
if radar_station=='L':
    radar_lat = 46.0401
    radar_lon = 8.8334
    radar_alt = 1627 
if radar_station=='P':
    radar_lat = 46.37083
    radar_lon = 7.48694
    radar_alt = 2927
if radar_station=='W':
    radar_lat = 46.832
    radar_lon = 9.796
    radar_alt = 2892 
if radar_station=='A':
    radar_lat = 47.2841
    radar_lon = 8.5117
    radar_alt = 938 
if radar_station=='D':
    radar_lat = 46.425676
    radar_lon = 6.100092
    radar_alt = 1670

# Coordinates of cities
locarno_lat = 46.1700
locarno_lon = 8.7996
lugano_lat = 46.0037
lugano_lon = 8.9511

# Load radar data
path = '/scratch/mch/fackerma/orders/case_130721'

file = glob.glob(path + '/MLL/ML' + radar_station + '21194' + valid_time3 + '0U.00' + elevation_scan)
file2 = glob.glob(path +'/DVL/srn/data/tmp/mof/DV' + radar_station + '/DV' + radar_station +'21194' + valid_time3 + '7L.80' + elevation_scan) # Dealiased velocity
file3 = glob.glob(path +'/HZC/HZC21194' + valid_time3 + 'VL.801') # Freezing level height composite
file4 = glob.glob(path +'/VAL/VA' + radar_station + '21194' + valid_time3 + '7U.019') # Vertical wind profile above radar
file5 = glob.glob(path +'/BZC/BZC21194' + valid_time3 + 'VL.845') # POH
file6 = glob.glob(path +'/MZC/MZC21194' + valid_time3 + 'VL.850') # MESH
file7 = glob.glob(path +'/TRTC/CZC21194' + valid_time3 + '0T.rdt') # Thunderstorm radar tracking; or .rdt .trt
file8 = glob.glob(path +'/CZC/CZC21194' + valid_time3 + 'VL.801') # Reflectivity composite
file9 = glob.glob(path +'/LZC/LZC21194' + valid_time3 + 'VL.801') # VIL composite
file10 = glob.glob(path +'/EZC/EZC21194' + valid_time3 + 'VL.820') # Echotops composite: 815, 820, 845, 850
file11 = glob.glob(path +'/OZC/OZC21194' + valid_time3 + 'VL.830') # CAPPI composite, ranging from 810 to 980 in steps of 10
file12 = glob.glob(path + '/dARC/ARC211940000FF.1440') # daily bias corrected rain accumulation
file13 = glob.glob(path + '/dBZC/BZC211942400VL.845') # daily POH
file14 = glob.glob(path + '/dRZC/RZC211942400VL.801') # daily Rain Rate
file15 = glob.glob(path + '/dCZC/CZC211942400VL.801') # daily Max Echo


# Read the data for the plots
data = pyart.aux_io.read_metranet(file[0], reader='python')
velocity = data.get_field(0, 'velocity').data
reflectivity = data.get_field(0, 'reflectivity').data

dv_data = pyart.aux_io.read_file_py(file2[0], physic_value=False)
dv_digital = dv_data.data
dv_header = dv_data.header
nyq = float(dv_header['nyquist'])
dv_velocity = transform_from_digital(dv_digital, nyq)

hzc_data = radlib.read_file(file3[0],physic_value=True)
hzc=hzc_data.data

dARC_data = radlib.read_file(file12[0],physic_value=True)
dARC=dARC_data.data

dBZC_data = radlib.read_file(file13[0],physic_value=True)
dBZC=dBZC_data.data

dRZC_data = radlib.read_file(file14[0],physic_value=True)
dRZC=dRZC_data.data

dCZC_data = radlib.read_file(file15[0],physic_value=True)
dCZC=dCZC_data.data

diff_reflectivity = data.get_field(0, 'differential_reflectivity').data

diff_phase = data.get_field(0, 'uncorrected_differential_phase').data

cc_ratio = data.get_field(0, 'uncorrected_cross_correlation_ratio').data

mzc_data = radlib.read_file(file6[0],physic_value=True)
mzc=mzc_data.data

bzc_data = radlib.read_file(file5[0],physic_value=True)
bzc=bzc_data.data

specw = data.get_field(0, 'spectrum_width').data

czc_data = radlib.read_file(file8[0],physic_value=True)
czc=czc_data.data

lzc_data = radlib.read_file(file9[0],physic_value=True)
lzc=lzc_data.data

ezc_data = radlib.read_file(file10[0],physic_value=True)
ezc=ezc_data.data

ozc_data = radlib.read_file(file11[0],physic_value=True)
ozc=ozc_data.data


path_trtc = {"lomdata": "/scratch/mch/fackerma/orders/case_130721/"}  # Replace with your actual path
trt_df, cells_list, timelist = read_TRT(path_trtc, ttime=valid_time3)
# Extract the first element from the list of cells
cells = cells_list[0]
# Define Swiss CH1903 to WGS84 transformer
transformer_trtc = Transformer.from_crs("epsg:21781", "epsg:4326", always_xy=True)
# Grid dimensions and origin
o_x, o_y = 255000, -160000  # Origin in Swiss coordinates
lx, ly = 710, 640           # Grid size (width x height)
cell_size = 1000            # 1 km grid spacing
# Create Swiss grid (CH1903 coordinates)
x_coords = np.linspace(o_x, o_x + (lx - 1) * cell_size, lx)
y_coords = np.linspace(o_y, o_y + (ly - 1) * cell_size, ly)
x_grid, y_grid = np.meshgrid(x_coords, y_coords)
# Convert grid to WGS84 (longitude, latitude)
lon_trtc, lat_trtc = transformer_trtc.transform(x_grid, y_grid)

#vad = read_VAD(file4[0])

# Convert echotop18 from km to meters
ezc_m = ezc * 1000
# Compute the wind gust estimator 'wge' (Trefalt 2017)
wge = np.sqrt((-3.1 * 10**-6 * (ezc**2)) + (20.6 * lzc))


resolution = 0.5
range_max = reflectivity.shape[1] * resolution
azimuths = np.linspace(0, 360, reflectivity.shape[0])
ranges = np.arange(0, range_max, resolution)

# Kdp own method
smoothed_diff_phase = gaussian_filter1d(diff_phase, sigma=2, axis=1) 
Kdp = np.gradient(smoothed_diff_phase, ranges, axis=1)

# KDP Vulpiani Method
Kdp = pyart.retrieve.kdp_vulpiani(data, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
Kdp = Kdp[0]['data']

llsd_vel = llsd(vel=dv_velocity, az_min=3, az_max=90, w_k=5, r_k=5, resolution=0.5)




# Transform radar polar coordinates to lat/lon
lats, lons = radar_to_latlon(radar_lat, radar_lon, ranges, azimuths)

# Define the Swiss grid (values based on metadata)
chx = np.arange(255000, 255000 + 710 * 1000 + 1, 1000)
chy = sorted(np.arange(-160000, -160000 + 640 * 1000 + 1, 1000), reverse=True)
X, Y = np.meshgrid(chx, chy)
# Initialize transformer for Swiss grid 
transformer = Transformer.from_crs(21781, 3035, always_xy=True)

#lons,lats = transformer.transform(X, Y)
clons, clats = transformer.transform(X, Y)
#lons, lats = transformer.transform(X, Y)

cmap_czc = LinearSegmentedColormap.from_list("custom_cmap", ["white", "blue", "green", "yellow","red","magenta","black"], N=256)
czc_levels = np.arange(-10, 65, 5)  # Steps of 2 for Reflectivity
czc_norm = BoundaryNorm(boundaries=czc_levels, ncolors=256, clip=True)
# Define color steps for Reflectivity and Velocity/Dealiased Velocity
reflectivity_levels = np.arange(-5, 65, 5)  # Steps of 2 for Reflectivity
velocity_levels = np.arange(-25, 26, 5)  # Steps of 1 for Velocity and Dealiased Velocity
hzc_levels = np.arange(1, 10, 1)
diff_reflectivity_levels = np.arange(-1,3.5,0.5)
diff_phase_levels = np.arange(-70,90,20)
cc_ratio_levels = np.arange(0.7,1,0.05)
mzc_levels = np.arange(0,5.5,0.5)
bzc_levels = np.arange(0,101,10)
specw_levels = np.arange(0,7,1)
Kdp_levels = np.arange(0,5.5,0.5)
lzc_levels = np.arange(2.5,25,2.5)
ezc_levels = np.arange(2,16,2)
wge_levels = np.arange(2,20,2)
az_shear_levels = np.arange(-10,12,2)
div_shear_levels = np.arange(-10,12,2)
# Create boundary norms
reflectivity_norm = BoundaryNorm(boundaries=reflectivity_levels, ncolors=plt.cm.inferno.N, clip=True)
velocity_norm = BoundaryNorm(boundaries=velocity_levels, ncolors=256, clip=True)
hzc_norm = BoundaryNorm(boundaries=hzc_levels, ncolors=256, clip=True)
diff_reflectivity_norm = BoundaryNorm(boundaries=diff_reflectivity_levels, ncolors=256, clip=True)
diff_phase_norm = BoundaryNorm(boundaries=diff_phase_levels, ncolors=256, clip=True)
cc_ratio_norm = BoundaryNorm(boundaries=cc_ratio_levels, ncolors=256, clip=True)
mzc_norm = BoundaryNorm(boundaries=mzc_levels, ncolors=256, clip=True)
bzc_norm = BoundaryNorm(boundaries=bzc_levels, ncolors=256, clip=True)
specw_norm = BoundaryNorm(boundaries=specw_levels, ncolors=256, clip=True)
Kdp_norm = BoundaryNorm(boundaries=Kdp_levels, ncolors=256, clip=True)
lzc_norm = BoundaryNorm(boundaries=lzc_levels, ncolors=256, clip=True)
ezc_norm = BoundaryNorm(boundaries=ezc_levels, ncolors=256, clip=True)
wge_norm = BoundaryNorm(boundaries=wge_levels, ncolors=256, clip=True)
az_shear_norm = BoundaryNorm(boundaries=az_shear_levels, ncolors=256, clip=True)
div_shear_norm = BoundaryNorm(boundaries=div_shear_levels, ncolors=256, clip=True)

cmap_velocity = LinearSegmentedColormap.from_list("custom_cmap", ["blue", "white", "red"], N=256)
cmap_hzc = LinearSegmentedColormap.from_list("custom_cmap", ["white", "dodgerblue", "midnightblue"], N=256)
cmap_diff_reflectivity = LinearSegmentedColormap.from_list("custom_cmap", ["blue", "white","red"], N=256)
cmap_diff_phase = LinearSegmentedColormap.from_list("custom_cmap", ["blue", "white","red"], N=256)
cmap_cc_ratio = LinearSegmentedColormap.from_list("custom_cmap", ["darkblue", "lightblue","yellow","orange","red"], N=256)
cmap_Kdp = LinearSegmentedColormap.from_list("custom_cmap", ["darkblue", "lightblue","yellow","orange","red"], N=256)
cmap_bzc = LinearSegmentedColormap.from_list("custom_cmap", ["black","yellow", "red"], N=256)
cmap_specw = LinearSegmentedColormap.from_list("custom_cmap", ["darkblue", "lightblue","yellow","orange","red"], N=256)
cmap_lzc = LinearSegmentedColormap.from_list("custom_cmap", ["black", "cyan", "purple"], N=256)
cmap_ezc = plt.cm.Blues
cmap_wge = plt.cm.magma
cmap_vad = plt.cm.Accent
cmap_az_shear = LinearSegmentedColormap.from_list("custom_cmap", ["aqua","blue", "black", "red","yellow"], N=256)
cmap_div_shear = cmap_az_shear

# Assess rhi files
files_rhi = sorted(glob.glob(path + '/MLL/MLL21194' + valid_time3 + '*')) # Read all elevations for rhi
for i,f in enumerate(files_rhi):
    radar = pyart.aux_io.read_metranet(f)
    
    if i == 0:
        radar_merged = radar
    else:
        radar_merged = pyart.util.join_radar(radar_merged, 
                                       radar)
#corr_vel = pyart.correct.dealias_region_based(radar_merged)
#radar_merged.add_field('corrected_velocity', corr_vel)



# Data import, assimilation
file_path = '/scratch/mch/fackerma/orders/case_130721/stations/20210713.csv'

# Open the file and read its contents
with open(file_path, 'r') as file:
    station_data = file.readlines()  # Read all lines into a list

# Import stations readme
file_path = '/users/fackerma/newproject1/stations_readme.csv'

# Open and read the file as plain text
try:
    with open(file_path, 'r') as stations:
        stations = stations.readlines()  # Read all lines into a list
    #for line in content[:10]:  # Print the first 10 lines for inspection
        #print(line.strip())
except Exception as e:
    print(f"Error reading file: {e}")

# Parse the station metadata to create a dictionary mapping station ID to coordinates and altitude
station_info = {}

# Extract the relevant station data from 'stations' (first 158 rows)
for line in stations[0:0+158]:  # Starts from line 5, inclusive
    parts = line.strip().split("\t")
    station_id = parts[0].strip()  # First column: Station ID
    
    # Extract coordinates (last column)
    coordinates = parts[-1].strip()  
    x, y = map(int, coordinates.split("/"))  # Split the coordinates into x and y

    # Extract altitude (using regex to find the number before "m a.s.l.")
    altitude_match = re.search(r'(\d+)\s*m\s*a\.s\.l\.', parts[1])
    altitude = int(altitude_match.group(1)) if altitude_match else None  # Default to None if no altitude found
    
    station_info[station_id] = (x, y, altitude)

# Add Swiss coordinates and altitude to 'station_data'
updated_station_data = []

# Add headers for clarity
header = station_data[0].strip() + "   X_COORD   Y_COORD   ALTITUDE\n"
updated_station_data.append(header)

for line in station_data[1:]:  # Skip the header
    parts = line.strip().split()
    station_id = parts[0].strip()  # First column: Station ID
    
    # Lookup coordinates and altitude for the station
    x, y, altitude = station_info.get(station_id, (None, None, None))
    
    if x is not None and y is not None and altitude is not None:
        # Append the coordinates and altitude as new columns
        updated_line = line.strip() + f"   {x}   {y}   {altitude}\n"
    else:
        # In case no coordinates or altitude are found (unlikely here), keep it unchanged
        updated_line = line.strip() + "   N/A   N/A   N/A\n"
    
    updated_station_data.append(updated_line)

# Assuming converter.LV03toWGS84() function is defined, and it converts LV03 to WGS84
# Function to convert LV03 to WGS84 (assuming this is already defined)
# Example: converter.LV03toWGS84(x, y, altitude) returns [latitude, longitude, altitude]

# Iterate over each row in updated_station_data (skip the header)
for i, line in enumerate(updated_station_data[1:], start=1):
    parts = line.strip().split()
    
    # Extract the X_COORD, Y_COORD, and ALTITUDE for the current station
    x_coord = int(parts[-3])  # X_COORD is the second last column
    y_coord = int(parts[-2])  # Y_COORD is the last column before altitude
    altitude = int(parts[-1])  # ALTITUDE is the last column in the row
    
    # Convert LV03 (x_coord, y_coord, altitude) to WGS84 (latitude, longitude, altitude)
    wgs84 = converter.LV03toWGS84(x_coord, y_coord, altitude)
    
    # Append the WGS84 values as new columns: latitude, longitude, altitude
    updated_station_data[i] = line.strip() + f"   {wgs84[0]}   {wgs84[1]}   {wgs84[2]}"
    
# Add headers for latitude, longitude, and altitude to the header row
header = updated_station_data[0].strip() + "   LATITUDE   LONGITUDE   ALTITUDE_NEW"
updated_station_data[0] = header

# Data assimilation
# Extract the headers and rows
headers = updated_station_data[0].split()
data = [line.split() for line in updated_station_data[1:]]

# Create a DataFrame
df = pd.DataFrame(data, columns=headers)

# Convert relevant columns to numeric (e.g., the wind gust column '101')
df['101'] = pd.to_numeric(df['101'], errors='coerce')
df['197'] = pd.to_numeric(df['197'], errors='coerce')
df['UTC'] = (df['JAHR'].astype(str) +
                      df['MO'].astype(str).str.zfill(2) +
                      df['TG'].astype(str).str.zfill(2) +
                      df['HH'].astype(str).str.zfill(2) +
                      df['MM'].astype(str).str.zfill(2))

# Filter the data for the timestamp closest to 14:30 UTC (1430)
target_time = '20210713' + valid_time3
filtered_df = df[df['UTC'] == target_time]
# Filter out rows where the '101' value is greater than 200
filtered_df = filtered_df[filtered_df['101'] <= 200]


# Extract necessary columns (station ID, wind speed, wind direction)
wind_speeds = filtered_df['101'].values
wind_directions = filtered_df['197'].values
station_ids = filtered_df['STA'].values

filtered_df['LONGITUDE'] = pd.to_numeric(filtered_df['LONGITUDE'], errors='coerce')
filtered_df['LATITUDE'] = pd.to_numeric(filtered_df['LATITUDE'], errors='coerce')
filtered_df['101'] = pd.to_numeric(filtered_df['101'], errors='coerce')
filtered_df['197'] = pd.to_numeric(filtered_df['197'], errors='coerce')

# Drop rows with invalid data
filtered_df = filtered_df.dropna(subset=['LONGITUDE', 'LATITUDE', '101', '197'])






# Extents for plotting
extent_switzerland = [5.77, 10.8, 45.5, 48]
max_extent = [4, 12, 44, 49]
TicinoLarge_extent = [5.5, 10.5, 44.6, 47]


WGS84: 
[46.54620053012867, 7.017760746224894, 2024.97379732]
LV03: 
[567719, 155077, 1974]
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
py_decoder DBG verb=0
py_decoder DBG verbl=0
swap product table data ......
py_decoder DBG verb=0
py_decoder DBG verbl=0


/tmp/ipykernel_16304/3911758668.py:236: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  headers=pd.read_csv(file).iloc[7:8].iloc[0][0].split()
/tmp/ipykernel_16304/3911758668.py:240: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trt_df.loc[n,'traj_ID']=int(t[0].values)
/tmp/ipykernel_16304/3911758668.py:241: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trt_df.loc[n,'time']=int(t[1].values)
/tmp/ipykernel_16304/3911758668.py:261: DeprecationWarning: Conver

In [27]:
#	STA	JAHR	MO	TG	HH	MM	197	196	101	733	91	967	194	X_COORD	Y_COORD	ALTITUDE	LATITUDE	LONGITUDE	ALTITUDE_NEW	UTC
#22080	74	2021	7	13	6	40	149	12.9	32.3	37.0	4.6	1020.3	0.4	783155	143525	3294	46.41804658419139	9.821299032530126	3342.520841	202107130640
#20685	87	2021	7	13	15	50	263	15.0	31.6	22.1	6.5	1011.1	6.1	704434	117458	1661	46.20047564456724	8.791656743493647	1711.10288248	202107131550
#22091	74	2021	7	13	8	30	147	16.2	30.1	36.5	4.1	1019.9	0.7	783155	143525	3294	46.41804658419139	9.821299032530126	3342.520841	202107130830
#19621	414	2021	7	13	7	40	174	21.1	30.0	28.9	7.8	1013.0	7.7	714263	140944	2171	46.410112342195	8.924731122936562	2220.44731404	202107130740
#6827	60	2021	7	13	2	0	150	21.4	29.9	31.1	9.1	1015.8	7.1	690050	167475	2286	46.65243510758286	8.615046317179816	2335.1517360000003	202107130200

In [ ]:
# Multi-variable plot
projection = ccrs.epsg(3035)
# Create figure with three subplots (Reflectivity, Velocity, and Dealiased Velocity)
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(24,24), subplot_kw={'projection': projection})

# Set background to black for all subplots
for row in axs:  # axs is a 2D array, so iterate through rows
    for ax in row:  # Iterate through each subplot in the row
        fig.patch.set_facecolor('black')  # Set figure background
        ax.set_facecolor('black')  # Set axis background



# Plot Reflectivity composite
cmap_czc = plt.cm.inferno
cmap_czc.set_bad(color='grey')
p1 = axs[0,0].pcolormesh(clons, clats, czc, vmin=0, vmax=55, 
                   cmap=cmap_czc, transform=projection)
# Add a colorbar
cbar1 = plt.colorbar(p1, ax=axs[0,0], extend='both', cmap=cmap_czc, norm=czc_norm, boundaries=czc_levels, ticks=czc_levels
                     )
cbar1.outline.set_edgecolor('white')
cbar1.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color='white')
axs[0,0].set_title('Z Composite [dBZ]', color='white', loc='right')


# Plot Dealiased Velocity
cmap_velocity = LinearSegmentedColormap.from_list("custom_cmap", ["blue", "white", "red"], N=256)
cmap_velocity.set_bad(color='black')
# Assuming lons and lats are 1D arrays
p2 = axs[0,1].pcolormesh(lons, lats, dv_velocity, cmap=cmap_velocity, norm=velocity_norm, transform=ccrs.PlateCarree())
cbar2 = plt.colorbar(p2, ax=axs[0,1], cmap=cmap_velocity, extend='both', boundaries=velocity_levels, ticks=velocity_levels)
cbar2.outline.set_edgecolor('white')
cbar2.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar2.ax.axes, 'yticklabels'), color='white')
axs[0,1].set_title('Velocity [m s⁻¹]', color='white', loc='right')


# Plot Freezing level height
# Mask values of freezing level height that are 0 or lower
hzc_masked = np.ma.masked_less_equal(hzc, 0)
# Define a custom colormap, setting the under value (for masked/low values) to black
cmap_hzc = LinearSegmentedColormap.from_list("custom_cmap", ["white", "dodgerblue", "midnightblue"], N=256)
#cmap_hzc.set_under('black')  # Black for values 0 or lower
cmap_hzc.set_bad(color='black')  # Black for invalid data
# Plot the freezing level height with the masked values
p3 = axs[0,2].pcolormesh(clons, clats, hzc_masked, vmin=0, vmax=12, 
                         cmap=cmap_hzc, transform=projection)
# Add a colorbar
cbar3 = plt.colorbar(p3, ax=axs[0,2], extend='both', cmap=cmap_hzc, boundaries=hzc_levels, ticks=hzc_levels)
cbar3.outline.set_edgecolor('white')
cbar3.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar3.ax.axes, 'yticklabels'), color='white')
axs[0,2].set_title('Freezing Level Height Composite [km]', color='white', loc='right')


# Plot Differential Reflectivity
cmap_diff_reflectivity = LinearSegmentedColormap.from_list("custom_cmap", ["blue", "white","red"], N=256)
cmap_diff_reflectivity.set_bad(color='black')
p4 = axs[1,0].pcolormesh(lons, lats, diff_reflectivity, cmap=cmap_diff_reflectivity, norm=diff_reflectivity_norm, 
                         transform=ccrs.PlateCarree())
cbar4 = plt.colorbar(p4, ax=axs[1,0], cmap=cmap_diff_reflectivity, extend='both', boundaries=diff_reflectivity_levels, ticks=diff_reflectivity_levels
                     )
cbar4.outline.set_edgecolor('white')
cbar4.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar4.ax.axes, 'yticklabels'), color='white')
axs[1,0].set_title('Zdr [dB]', color='white', loc='right')


# Plot Differential Phase
cmap_diff_phase = LinearSegmentedColormap.from_list("custom_cmap", ["blue", "white","red"], N=256)
cmap_diff_phase.set_bad(color='black')
p5 = axs[1,1].pcolormesh(lons, lats, diff_phase, cmap=cmap_diff_phase, norm=diff_phase_norm, 
                         transform=ccrs.PlateCarree())
cbar5 = plt.colorbar(p5, ax=axs[1,1], cmap=cmap_diff_phase, extend='both', boundaries=diff_phase_levels, ticks=diff_phase_levels
                     )
cbar5.outline.set_edgecolor('white')
cbar5.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar5.ax.axes, 'yticklabels'), color='white')
axs[1,1].set_title('Uncorrected ΦDP [°]', color='white', loc='right')


# Plot CC ratio
cmap_cc_ratio = LinearSegmentedColormap.from_list("custom_cmap", ["darkblue", "lightblue","yellow","orange","red"], N=256)
cmap_cc_ratio.set_bad(color='black')
p6 = axs[1,2].pcolormesh(lons, lats, cc_ratio, cmap=cmap_cc_ratio, norm=cc_ratio_norm, 
                         transform=ccrs.PlateCarree())
cbar6 = plt.colorbar(p6, ax=axs[1,2], cmap=cmap_cc_ratio, extend='both', boundaries=cc_ratio_levels, ticks=cc_ratio_levels
                     )
cbar6.outline.set_edgecolor('white')
cbar6.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar6.ax.axes, 'yticklabels'), color='white')
axs[1,2].set_title('Uncorrected ρhv', color='white', loc='right')


# Plot Kdp
cmap_Kdp = LinearSegmentedColormap.from_list("custom_cmap", ["darkblue", "lightblue","yellow","orange","red"], N=256)
cmap_Kdp.set_bad(color='black')
p7 = axs[2,0].pcolormesh(lons, lats, Kdp, norm=Kdp_norm,
                   cmap=cmap_Kdp, transform=ccrs.PlateCarree())
# Add a colorbar
cbar7 = plt.colorbar(p7, ax=axs[2,0], extend='both', cmap=cmap_Kdp, boundaries=Kdp_levels, ticks=Kdp_levels
                     )
cbar7.outline.set_edgecolor('white')
cbar7.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar7.ax.axes, 'yticklabels'), color='white')
axs[2,0].set_title('Kdp [° km⁻¹]', color='white', loc='right')


# Plot POH
cmap_bzc = LinearSegmentedColormap.from_list("custom_cmap", ["black","yellow", "red"], N=256)
cmap_bzc.set_bad(color='black')
p8 = axs[2,1].pcolormesh(clons, clats, bzc, norm=bzc_norm,
                   cmap=cmap_bzc, transform=projection)
# Add a colorbar
cbar8 = plt.colorbar(p8, ax=axs[2,1], extend='both', cmap=cmap_bzc, boundaries=bzc_levels, ticks=bzc_levels
                     )
cbar8.outline.set_edgecolor('white')
cbar8.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar8.ax.axes, 'yticklabels'), color='white')
axs[2,1].set_title('POH Composite [%]', color='white', loc='right')


# Plot Spectrum Width
cmap_specw = LinearSegmentedColormap.from_list("custom_cmap", ["darkblue", "lightblue","yellow","orange","red"], N=256)
cmap_specw.set_bad(color='black')
p9 = axs[2,2].pcolormesh(lons, lats, specw, cmap=cmap_specw, norm=specw_norm, 
                         transform=ccrs.PlateCarree())
cbar9 = plt.colorbar(p9, ax=axs[2,2], cmap=cmap_specw, extend='both', boundaries=specw_levels, ticks=specw_levels
                     )
cbar9.outline.set_edgecolor('white')
cbar9.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar9.ax.axes, 'yticklabels'), color='white')
axs[2,2].set_title('Spectrum Width [m s⁻¹]', color='white', loc='right')

# Plot VIL composite
#lzc = np.ma.masked_less_equal(lzc, 0.1)
cmap_lzc = LinearSegmentedColormap.from_list("custom_cmap", ["black", "cyan", "purple"#, "red"
                                                             ], N=256)
cmap_lzc.set_bad(color='black')
p10 = axs[3,0].pcolormesh(clons, clats, lzc, vmin=1, vmax=22.5, 
                   cmap=cmap_lzc, transform=projection)
# Add a colorbar
cbar10 = plt.colorbar(p10, ax=axs[3,0], extend='both', cmap=cmap_lzc, norm=lzc_norm, boundaries=lzc_levels, ticks=lzc_levels
                     )
cbar10.outline.set_edgecolor('white')
cbar10.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar10.ax.axes, 'yticklabels'), color='white')
axs[3,0].set_title('VIL Composite [kg m⁻²]', color='white', loc='right')

# Plot Echotop 20 composite
# Set a mask
ezc = np.ma.masked_less_equal(ezc, 0)
cmap_ezc = plt.cm.Blues
cmap_ezc.set_bad(color='black')
p11 = axs[3,1].pcolormesh(clons, clats, ezc, vmin=0, vmax=14,  
                   cmap=cmap_ezc, transform=projection)
# Add a colorbar
cbar11 = plt.colorbar(p11, ax=axs[3,1], extend='both', cmap=cmap_ezc, norm=ezc_norm, boundaries=ezc_levels, ticks=ezc_levels
                     )
cbar11.outline.set_edgecolor('white')
cbar11.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar11.ax.axes, 'yticklabels'), color='white')
axs[3,1].set_title('Echotop20 Composite [km]', color='white', loc='right')

# Plot wge composite
# Set a mask
wge = np.ma.masked_less_equal(wge, 0)
cmap_wge = plt.cm.magma
cmap_wge.set_bad(color='black')
p12 = axs[3,2].pcolormesh(clons, clats, wge, vmax=18, 
                   cmap=cmap_wge, transform=projection)
# Add a colorbar
cbar12 = plt.colorbar(p12, ax=axs[3,2], extend='both', cmap=cmap_wge, norm=wge_norm, boundaries=wge_levels, ticks=wge_levels
                     )
cbar12.outline.set_edgecolor('white')
cbar12.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar12.ax.axes, 'yticklabels'), color='white')
axs[3,2].set_title('Max. Downdraft Velocity w [m s⁻¹]', color='white', loc='right')

# (Plotting of wind barbs and other features remains the same)
# Add Radar, Geneva, and Lausanne locations to all plots
for row in axs:
    for ax in row:
            #ax.plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree())  # Radar location
        ax.plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree(),
        path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
        ax.plot(locarno_lon, locarno_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())  # Geneva
        ax.plot(lugano_lon, lugano_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())  # Lausanne
        ax.set_extent([7.6, 9.75 , 45.35, 46.8], crs=ccrs.PlateCarree())
         # Add borders, coastlines, and lakes with custom contour lines
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7)
        ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
        ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)
        # Add city names with a black outline for visibility
        locarno_text = ax.text(locarno_lon + 0.04, locarno_lat, 'Locarno', color='white', fontsize=14, transform=ccrs.PlateCarree())
        lugano_text = ax.text(lugano_lon + 0.04, lugano_lat, 'Lugano', color='white', fontsize=14, transform=ccrs.PlateCarree())
        locarno_text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])
        lugano_text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])

    



# Adjust margins and layout
#plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.3, wspace=0.3)
#plt.tight_layout(pad=2.0)

# Set main title
plt.suptitle(valid_time3 + ' UTC, Lema elevation scan: ' + elevation_scan #+ ', Barbs: 10 min mean surface wind [km h⁻¹]'
             , color='white', fontsize=16, x=0.5)
# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save the figure
#plt.savefig("/users/fackerma/newproject1/figures/second_case/" + valid_time3 + "_" + elevation_scan + "_all_variables.png", facecolor='black', dpi=300)

In [ ]:
# Multi-variable plot
projection = ccrs.epsg(3035)
# Create figure with three subplots (Reflectivity, Velocity, and Dealiased Velocity)
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(24, 24), subplot_kw={'projection': projection})

# Set background to black for all subplots
for row in axs:  # axs is a 2D array, so iterate through rows
    for ax in row:  # Iterate through each subplot in the row
        fig.patch.set_facecolor('black')  # Set figure background
        ax.set_facecolor('black')  # Set axis background


# Plot Reflectivity composite
cmap_czc.set_bad(color='grey')
p1 = axs[0,0].pcolormesh(clons, clats, czc, vmin=0, vmax=55, 
                   cmap=cmap_czc, transform=projection)
# Add a colorbar
cbar1 = plt.colorbar(p1, ax=axs[0,0], extend='both', cmap=cmap_czc, norm=czc_norm, boundaries=czc_levels, ticks=czc_levels
                     )
cbar1.outline.set_edgecolor('white')
cbar1.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color='white')
axs[0,0].set_title('Z Composite [dBZ]', color='white', loc='right')


# Plot Dealiased Velocity
cmap_velocity.set_bad(color='black')
p2 = axs[0,1].pcolormesh(lons, lats, dv_velocity, cmap=cmap_velocity, norm=velocity_norm, transform=ccrs.PlateCarree())
cbar2 = plt.colorbar(p2, ax=axs[0,1], cmap=cmap_velocity, extend='both', boundaries=velocity_levels, ticks=velocity_levels)
cbar2.outline.set_edgecolor('white')
cbar2.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar2.ax.axes, 'yticklabels'), color='white')
axs[0,1].set_title('Velocity [m s⁻¹]', color='white', loc='right')


# Plot Radial shear
#cmap_specw.set_bad(color='black')
p9 = axs[1,1].pcolormesh(lons, lats, llsd_vel[1], cmap=cmap_div_shear, norm=div_shear_norm, 
                         transform=ccrs.PlateCarree())
cbar9 = plt.colorbar(p9, ax=axs[1,1], cmap=cmap_div_shear, extend='both', boundaries=div_shear_levels, ticks=div_shear_levels
                     )
cbar9.outline.set_edgecolor('white')
cbar9.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar9.ax.axes, 'yticklabels'), color='white')
axs[1,1].set_title('Radial Shear', color='white', loc='right')

# Plot Azmiutal shear
#cmap_specw.set_bad(color='black')
p9 = axs[1,0].pcolormesh(lons, lats, llsd_vel[0], cmap=cmap_az_shear, norm=az_shear_norm, 
                         transform=ccrs.PlateCarree())
cbar9 = plt.colorbar(p9, ax=axs[1,0], cmap=cmap_az_shear, extend='both', boundaries=az_shear_levels, ticks=az_shear_levels
                     )
cbar9.outline.set_edgecolor('white')
cbar9.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar9.ax.axes, 'yticklabels'), color='white')
axs[1,0].set_title('Azimuthal Shear', color='white', loc='right')


# Plot Freezing level height
# Mask values of freezing level height that are 0 or lower
hzc_masked = np.ma.masked_less_equal(hzc, 0)
# Define a custom colormap, setting the under value (for masked/low values) to black
#cmap_hzc.set_under('black')  # Black for values 0 or lower
cmap_hzc.set_bad(color='black')  # Black for invalid data
# Plot the freezing level height with the masked values
p3 = axs[2,2].pcolormesh(clons, clats, hzc_masked, vmin=0, vmax=12, 
                         cmap=cmap_hzc, transform=projection)
# Add a colorbar
cbar3 = plt.colorbar(p3, ax=axs[2,2], extend='both', cmap=cmap_hzc, boundaries=hzc_levels, ticks=hzc_levels)
cbar3.outline.set_edgecolor('white')
cbar3.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar3.ax.axes, 'yticklabels'), color='white')
axs[2,2].set_title('Freezing Level Height Composite [km]', color='white', loc='right')


# Plot Differential Reflectivity
cmap_diff_reflectivity.set_bad(color='black')
p4 = axs[1,2].pcolormesh(lons, lats, diff_reflectivity, cmap=cmap_diff_reflectivity, norm=diff_reflectivity_norm, 
                         transform=ccrs.PlateCarree())
cbar4 = plt.colorbar(p4, ax=axs[1,2], cmap=cmap_diff_reflectivity, extend='both', boundaries=diff_reflectivity_levels, ticks=diff_reflectivity_levels
                     )
cbar4.outline.set_edgecolor('white')
cbar4.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar4.ax.axes, 'yticklabels'), color='white')
axs[1,2].set_title('Zdr [dB]', color='white', loc='right')


# Plot CC ratio
cmap_cc_ratio.set_bad(color='black')
p6 = axs[2,0].pcolormesh(lons, lats, cc_ratio, cmap=cmap_cc_ratio, norm=cc_ratio_norm, 
                         transform=ccrs.PlateCarree())
cbar6 = plt.colorbar(p6, ax=axs[2,0], cmap=cmap_cc_ratio, extend='both', boundaries=cc_ratio_levels, ticks=cc_ratio_levels
                     )
cbar6.outline.set_edgecolor('white')
cbar6.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar6.ax.axes, 'yticklabels'), color='white')
axs[2,0].set_title('Uncorrected ρhv', color='white', loc='right')


# Plot Kdp
cmap_Kdp.set_bad(color='black')
p7 = axs[0,2].pcolormesh(lons, lats, Kdp, norm=Kdp_norm,
                   cmap=cmap_Kdp, transform=ccrs.PlateCarree())
# Add a colorbar
cbar7 = plt.colorbar(p7, ax=axs[0,2], extend='both', cmap=cmap_Kdp, boundaries=Kdp_levels, ticks=Kdp_levels
                     )
cbar7.outline.set_edgecolor('white')
cbar7.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar7.ax.axes, 'yticklabels'), color='white')
axs[0,2].set_title('Kdp [° km⁻¹]', color='white', loc='right')


# Plot POH
cmap_bzc.set_bad(color='black')
p8 = axs[2,1].pcolormesh(clons, clats, bzc, norm=bzc_norm,
                   cmap=cmap_bzc, transform=projection)
# Add a colorbar
cbar8 = plt.colorbar(p8, ax=axs[2,1], extend='both', cmap=cmap_bzc, boundaries=bzc_levels, ticks=bzc_levels
                     )
cbar8.outline.set_edgecolor('white')
cbar8.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar8.ax.axes, 'yticklabels'), color='white')
axs[2,1].set_title('POH Composite [%]', color='white', loc='right')


# Plot VIL composite
#lzc = np.ma.masked_less_equal(lzc, 0.1)
cmap_lzc.set_bad(color='black')
p10 = axs[3,0].pcolormesh(clons, clats, lzc, vmin=1, vmax=22.5, 
                   cmap=cmap_lzc, transform=projection)
# Add a colorbar
cbar10 = plt.colorbar(p10, ax=axs[3,0], extend='both', cmap=cmap_lzc, norm=lzc_norm, boundaries=lzc_levels, ticks=lzc_levels
                     )
cbar10.outline.set_edgecolor('white')
cbar10.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar10.ax.axes, 'yticklabels'), color='white')
axs[3,0].set_title('VIL Composite [kg m⁻²]', color='white', loc='right')

# Plot Echotop 20 composite
# Set a mask
ezc = np.ma.masked_less_equal(ezc, 0)
cmap_ezc.set_bad(color='black')
p11 = axs[3,1].pcolormesh(clons, clats, ezc, vmin=0, vmax=14, 
                   cmap=cmap_ezc, transform=projection)
# Add a colorbar
cbar11 = plt.colorbar(p11, ax=axs[3,1], extend='both', cmap=cmap_ezc, norm=ezc_norm, boundaries=ezc_levels, ticks=ezc_levels
                     )
cbar11.outline.set_edgecolor('white')
cbar11.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar11.ax.axes, 'yticklabels'), color='white')
axs[3,1].set_title('Echotop20 Composite [km]', color='white', loc='right')

# Plot wge composite
# Set a mask
wge = np.ma.masked_less_equal(wge, 0)
cmap_wge.set_bad(color='black')
p12 = axs[3,2].pcolormesh(clons, clats, wge, vmax=18, 
                   cmap=cmap_wge, transform=projection)
# Add a colorbar
cbar12 = plt.colorbar(p12, ax=axs[3,2], extend='both', cmap=cmap_wge, norm=wge_norm, boundaries=wge_levels, ticks=wge_levels
                     )
cbar12.outline.set_edgecolor('white')
cbar12.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar12.ax.axes, 'yticklabels'), color='white')
axs[3,2].set_title('Max. Downdraft Velocity w [m s⁻¹]', color='white', loc='right')

# (Plotting of wind barbs and other features remains the same)
for row in axs:
    for ax in row:
            #ax.plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree())  # Radar location
        ax.plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree(),
        path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
        ax.plot(locarno_lon, locarno_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())  # Geneva
        ax.plot(lugano_lon, lugano_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())  # Lausanne
        ax.set_extent([7.6, 9.75 , 45.35, 46.8], crs=ccrs.PlateCarree())
         # Add borders, coastlines, and lakes with custom contour lines
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7)
        ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
        ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)
        # Add city names with a black outline for visibility
        locarno_text = ax.text(locarno_lon + 0.04, locarno_lat, 'Locarno', color='white', fontsize=14, transform=ccrs.PlateCarree())
        lugano_text = ax.text(lugano_lon + 0.04, lugano_lat, 'Lugano', color='white', fontsize=14, transform=ccrs.PlateCarree())
        locarno_text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])
        lugano_text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])





# Extract necessary columns (station ID, wind speed, wind direction)
wind_speeds = filtered_df['101'].values
wind_directions = filtered_df['197'].values
station_ids = filtered_df['STA'].values

filtered_df['LONGITUDE'] = pd.to_numeric(filtered_df['LONGITUDE'], errors='coerce')
filtered_df['LATITUDE'] = pd.to_numeric(filtered_df['LATITUDE'], errors='coerce')
filtered_df['101'] = pd.to_numeric(filtered_df['101'], errors='coerce')
filtered_df['197'] = pd.to_numeric(filtered_df['197'], errors='coerce')

# Drop rows with invalid data
filtered_df = filtered_df.dropna(subset=['LONGITUDE', 'LATITUDE', '101', '197'])


# Select a specific subplot for the wind barbs (e.g., the first subplot)
#ax = axs[0:3, 0:2]  # Adjust the index based on your layout preference

# Iterate over rows in the filtered dataframe
for _, row in filtered_df.iterrows():
    try:
        # Extract values directly without wrapping them in float or np.array
        longitude = row['LONGITUDE']
        latitude = row['LATITUDE']
        speed = row['101']  # Wind speed
        direction = row['197']  # Wind direction

        # Ensure the values are 1-dimensional arrays
        longitude = np.array([longitude])  # Wrap as 1D array
        latitude = np.array([latitude])    # Wrap as 1D array
        speed = np.array([speed])
        direction = np.array([direction])
        
        # Convert to wind components
        u = -speed * np.sin(np.radians(direction)) * 3.6
        v = -speed * np.cos(np.radians(direction)) * 3.6
        
        for row in axs:
            for ax in row:
        # Add a single wind barb
                ax.barbs(longitude, latitude, u, v, transform=ccrs.PlateCarree(), color='black', length=6, linewidth=1.5)
                ax.barbs(longitude, latitude, u, v, transform=ccrs.PlateCarree(), color='lime', length=6, linewidth=1)
    except ValueError as e:
        print(f"Skipping row due to error: {e}")


# Set main title
plt.suptitle(valid_time3 + ' UTC, ' + radar_name + ' elevation scan: ' + elevation_scan + ', Barbs: 10 min max s⁻¹ gust speed [km h⁻¹]', color='white', fontsize=16, x=0.5)
# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save the figure
#plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station + valid_time3 + "_" + elevation_scan + "_all_variables.png", facecolor='black', dpi=300)

In [ ]:
# Daily POH

projection = ccrs.epsg(3035)
# Create figure with one subplot (dBZC composite)
fig, ax = plt.subplots(figsize=(6, 5), subplot_kw={'projection': projection})

# Set background to black for the plot
fig.patch.set_facecolor('black')  # Set the overall figure background
ax.set_facecolor('black')  # Set axis background

# Plot dBZC composite
dBZC = np.ma.masked_less_equal(dBZC, 0)
cmap_dBZC = cmap_bzc
cmap_dBZC.set_bad(color='black')
p2 = ax.pcolormesh(clons, clats, dBZC, cmap=cmap_dBZC, transform=projection, vmin=0, vmax=100)

# Add a colorbar for the plot, capping it between 0 and 100
cbar2 = plt.colorbar(p2, ax=ax, extend='neither', cmap=cmap_dBZC, boundaries=np.linspace(0, 100, 11), ticks=np.linspace(0, 100, 11))
cbar2.outline.set_edgecolor('white')
cbar2.ax.yaxis.set_tick_params(color='white', labelsize=14)
plt.setp(plt.getp(cbar2.ax.axes, 'yticklabels'), color='white')
ax.set_title('POH [%]', color='white', loc='right', fontsize=14)

# Add Radar, Geneva, and Lausanne locations to the plot
# ax.plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree(),
#         path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
# ax.plot(geneva_lon, geneva_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())
# ax.plot(lausanne_lon, lausanne_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())
ax.set_extent([5, 11, 44.7, 48], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7#, transform=ccrs.PlateCarree()
               )
ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7#, transform=ccrs.PlateCarree()
               )
ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7#, transform=ccrs.PlateCarree()
               )
ax.gridlines(alpha=0.5)

# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save the figure (adjust this line to your desired path)
#plt.savefig("/users/fackerma/newproject1/figures/second_case/dailyPOH_2.png", facecolor='black', dpi=300)




## TRTC

In [ ]:
# Locarno 0700
downburst_location = 'Locarno'
downburst_x = 704167
downburst_y = 114317
downburst_z = 367

llsd_vel = llsd(vel=dv_velocity, az_min=3, az_max=30, w_k=2, r_k=1.5, resolution=0.5)


projection = ccrs.epsg(3035)
# Create figure with three subplots (Reflectivity, Velocity, and Dealiased Velocity)
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(24, 12), subplot_kw={'projection': projection})

# Set background to black for all subplots
for row in axs:  # axs is a 2D array, so iterate through rows
    for ax in row:  # Iterate through each subplot in the row
        fig.patch.set_facecolor('white')  # Set figure background
        ax.set_facecolor('white')  # Set axis background

edgecolor = 'black'

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

# Plot Dealiased Velocity
cmap_velocity.set_bad(color='black')
p2 = axs[0,0].pcolormesh(lons, lats, dv_velocity, cmap=cmap_velocity, norm=velocity_norm, transform=ccrs.PlateCarree())
cbar2 = plt.colorbar(p2, ax=axs[0,0], cmap=cmap_velocity, extend='both', boundaries=velocity_levels, ticks=velocity_levels)
cbar2.outline.set_edgecolor(edgecolor)
cbar2.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar2.ax.axes, 'yticklabels'), color=edgecolor)
axs[0,0].set_title('Velocity [m s⁻¹]', color=edgecolor, loc='right')


# Plot Azmiuthal shear
#cmap_specw.set_bad(color='black')
cmap_az_shear.set_bad(color='black')
p9 = axs[0,1].pcolormesh(lons, lats, llsd_vel[0], cmap=cmap_az_shear, norm=az_shear_norm, 
                         transform=ccrs.PlateCarree())
cbar9 = plt.colorbar(p9, ax=axs[0,1], cmap=cmap_az_shear, extend='both', boundaries=az_shear_levels, ticks=az_shear_levels
                     )
cbar9.outline.set_edgecolor(edgecolor)
cbar9.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar9.ax.axes, 'yticklabels'), color=edgecolor)
axs[0,1].set_title('Azimuthal Shear [m s⁻¹ km⁻¹]', color=edgecolor, loc='right')

# Plot Radial shear
#cmap_specw.set_bad(color='black')
cmap_div_shear.set_bad(color='black')
p9 = axs[0,2].pcolormesh(lons, lats, llsd_vel[1], cmap=cmap_div_shear, norm=div_shear_norm, 
                         transform=ccrs.PlateCarree())
cbar9 = plt.colorbar(p9, ax=axs[0,2], cmap=cmap_div_shear, extend='both', boundaries=div_shear_levels, ticks=div_shear_levels
                     )
cbar9.outline.set_edgecolor(edgecolor)
cbar9.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar9.ax.axes, 'yticklabels'), color=edgecolor)
axs[0,2].set_title('Radial Shear [m s⁻¹ km⁻¹]', color=edgecolor, loc='right')

# Plot Reflectivity composite
cmap_czc.set_bad(color='grey')
p1 = axs[1,0].pcolormesh(lons, lats, reflectivity, cmap=cmap_czc, norm=czc_norm, transform=ccrs.PlateCarree())
# Add a colorbar
cbar1 = plt.colorbar(p1, ax=axs[1,0], extend='both', cmap=cmap_czc, norm=czc_norm, boundaries=czc_levels, ticks=czc_levels
                     )
cbar1.outline.set_edgecolor(edgecolor)
cbar1.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color=edgecolor)
axs[1,0].set_title('Z [dBZ]', color=edgecolor, loc='right')


# Plot Kdp
cmap_Kdp.set_bad(color='black')
p7 = axs[1,1].pcolormesh(lons, lats, Kdp, norm=Kdp_norm,
                   cmap=cmap_Kdp, transform=ccrs.PlateCarree())
# Add a colorbar
cbar7 = plt.colorbar(p7, ax=axs[1,1], extend='both', cmap=cmap_Kdp, boundaries=Kdp_levels, ticks=Kdp_levels
                     )
cbar7.outline.set_edgecolor(edgecolor)
cbar7.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar7.ax.axes, 'yticklabels'), color=edgecolor)
axs[1,1].set_title('Kdp [° km⁻¹]', color=edgecolor, loc='right')


# Plot wge composite
# Set a mask
wge = np.ma.masked_less_equal(wge, 0)
cmap_wge.set_bad(color='black')
p12 = axs[1,2].pcolormesh(clons, clats, wge, vmax=18, 
                   cmap=cmap_wge, transform=projection)
# Add a colorbar
cbar12 = plt.colorbar(p12, ax=axs[1,2], extend='both', cmap=cmap_wge, norm=wge_norm, boundaries=wge_levels, ticks=wge_levels
                     )
cbar12.outline.set_edgecolor(edgecolor)
cbar12.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar12.ax.axes, 'yticklabels'), color=edgecolor)
axs[1,2].set_title('Max. Downdraft Velocity w [m s⁻¹]', color=edgecolor, loc='right')


# Select a specific subplot for the wind barbs (e.g., the first subplot)
#ax = axs[0:3, 0:2]  # Adjust the index based on your layout preference


# (Plotting of wind barbs and other features remains the same)
# Add Radar, Geneva, and Lausanne locations to all plots
for row in axs:
    for ax in row:
        # Radar location or points of interest
        ax.plot(downburst_lon, downburst_lat, marker='x', color='red', markersize=16, 
            transform=ccrs.PlateCarree(), 
            path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
    
        # Set the spatial extent of the plot
        ax.set_extent([downburst_lon - 0.4, downburst_lon + 0.4, downburst_lat -0.2, downburst_lat + 0.2], 
                  crs=ccrs.PlateCarree())
    
        # Add geographic features
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=1)
        ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=1)
        ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=1)

from matplotlib.colors import to_rgba
lime_green_with_alpha = to_rgba("lime", alpha=0.5)

# Add TRT cells contour to every subplot
for row in axs:
    for ax in row:
        # Ensure cells, lons, and lats are aligned and properly scaled
        ax.contour(lon_trtc, lat_trtc, cells, levels=np.arange(0, 11), colors=[lime_green_with_alpha], alpha=0.5, transform=ccrs.PlateCarree())

# Iterate over rows in the filtered dataframe
for _, row in filtered_df.iterrows():
    try:
        # Extract values directly without wrapping them in float or np.array
        longitude = row['LONGITUDE']
        latitude = row['LATITUDE']
        speed = row['101']  # Wind speed
        direction = row['197']  # Wind direction

        # Ensure the values are 1-dimensional arrays
        longitude = np.array([longitude])  # Wrap as 1D array
        latitude = np.array([latitude])    # Wrap as 1D array
        speed = np.array([speed])
        direction = np.array([direction])
        
        # Convert to wind components
        u = -speed * np.sin(np.radians(direction)) * 3.6
        v = -speed * np.cos(np.radians(direction)) * 3.6
        
        for row in axs:
            for ax in row:
        # Add a single wind barb
                ax.barbs(longitude, latitude, u, v, transform=ccrs.PlateCarree(), color='black', length=8, linewidth=2.5, zorder=10)
                ax.barbs(longitude, latitude, u, v, transform=ccrs.PlateCarree(), color='white', length=8, linewidth=1, zorder=11)
    except ValueError as e:
        print(f"Skipping row due to error: {e}")



# Set main title
plt.suptitle(valid_time3 + ' UTC, ' + radar_name + ' elevation scan: ' + elevation_scan + ', Barbs: 10 min max s⁻¹ gust speed [km h⁻¹], Contour: TRT Cell', color=edgecolor, fontsize=16, x=0.5)
# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0, 1, 0.95])

plt.savefig("/users/fackerma/newproject1/figures/second_case/TRT2/TRT_" + downburst_location + "_" + radar_station + "_" + valid_time3 + "_" + elevation_scan + "_2.png", dpi=300)

In [ ]:
# Lugano 0650
downburst_location = 'Lugano'
downburst_x = 717874
downburst_y = 95884
downburst_z = 273

projection = ccrs.epsg(3035)
# Create figure with three subplots (Reflectivity, Velocity, and Dealiased Velocity)
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(24, 12), subplot_kw={'projection': projection})

# Set background to black for all subplots
for row in axs:  # axs is a 2D array, so iterate through rows
    for ax in row:  # Iterate through each subplot in the row
        fig.patch.set_facecolor('white')  # Set figure background
        ax.set_facecolor('white')  # Set axis background

edgecolor = 'black'

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

# Plot Dealiased Velocity
cmap_velocity.set_bad(color='black')
p2 = axs[0,0].pcolormesh(lons, lats, dv_velocity, cmap=cmap_velocity, norm=velocity_norm, transform=ccrs.PlateCarree())
cbar2 = plt.colorbar(p2, ax=axs[0,0], cmap=cmap_velocity, extend='both', boundaries=velocity_levels, ticks=velocity_levels)
cbar2.outline.set_edgecolor(edgecolor)
cbar2.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar2.ax.axes, 'yticklabels'), color=edgecolor)
axs[0,0].set_title('Velocity [m s⁻¹]', color=edgecolor, loc='right')

# Plot Azmiuthal shear
#cmap_specw.set_bad(color='black')
cmap_az_shear.set_bad(color='black')
p9 = axs[0,1].pcolormesh(lons, lats, llsd_vel[0], cmap=cmap_az_shear, norm=az_shear_norm, 
                         transform=ccrs.PlateCarree())
cbar9 = plt.colorbar(p9, ax=axs[0,1], cmap=cmap_az_shear, extend='both', boundaries=az_shear_levels, ticks=az_shear_levels
                     )
cbar9.outline.set_edgecolor(edgecolor)
cbar9.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar9.ax.axes, 'yticklabels'), color=edgecolor)
axs[0,1].set_title('Azimuthal Shear [m s⁻¹ km⁻¹]', color=edgecolor, loc='right')

# Plot Radial shear
#cmap_specw.set_bad(color='black')
cmap_div_shear.set_bad(color='black')
p9 = axs[0,2].pcolormesh(lons, lats, llsd_vel[1], cmap=cmap_div_shear, norm=div_shear_norm, 
                         transform=ccrs.PlateCarree())
cbar9 = plt.colorbar(p9, ax=axs[0,2], cmap=cmap_div_shear, extend='both', boundaries=div_shear_levels, ticks=div_shear_levels
                     )
cbar9.outline.set_edgecolor(edgecolor)
cbar9.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar9.ax.axes, 'yticklabels'), color=edgecolor)
axs[0,2].set_title('Radial Shear [m s⁻¹ km⁻¹]', color=edgecolor, loc='right')

# Plot Reflectivity composite
cmap_czc.set_bad(color='grey')
p1 = axs[1,0].pcolormesh(lons, lats, reflectivity, cmap=cmap_czc, norm=czc_norm, transform=ccrs.PlateCarree())
# Add a colorbar
cbar1 = plt.colorbar(p1, ax=axs[1,0], extend='both', cmap=cmap_czc, norm=czc_norm, boundaries=czc_levels, ticks=czc_levels
                     )
cbar1.outline.set_edgecolor(edgecolor)
cbar1.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color=edgecolor)
axs[1,0].set_title('Z [dBZ]', color=edgecolor, loc='right')


# Plot Kdp
cmap_Kdp.set_bad(color='black')
p7 = axs[1,1].pcolormesh(lons, lats, Kdp, norm=Kdp_norm,
                   cmap=cmap_Kdp, transform=ccrs.PlateCarree())
# Add a colorbar
cbar7 = plt.colorbar(p7, ax=axs[1,1], extend='both', cmap=cmap_Kdp, boundaries=Kdp_levels, ticks=Kdp_levels
                     )
cbar7.outline.set_edgecolor(edgecolor)
cbar7.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar7.ax.axes, 'yticklabels'), color=edgecolor)
axs[1,1].set_title('Kdp [° km⁻¹]', color=edgecolor, loc='right')


# Plot wge composite
# Set a mask
wge = np.ma.masked_less_equal(wge, 0)
cmap_wge.set_bad(color='black')
p12 = axs[1,2].pcolormesh(clons, clats, wge, vmax=18, 
                   cmap=cmap_wge, transform=projection)
# Add a colorbar
cbar12 = plt.colorbar(p12, ax=axs[1,2], extend='both', cmap=cmap_wge, norm=wge_norm, boundaries=wge_levels, ticks=wge_levels
                     )
cbar12.outline.set_edgecolor(edgecolor)
cbar12.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar12.ax.axes, 'yticklabels'), color=edgecolor)
axs[1,2].set_title('Max. Downdraft Velocity w [m s⁻¹]', color=edgecolor, loc='right')


# Select a specific subplot for the wind barbs (e.g., the first subplot)
#ax = axs[0:3, 0:2]  # Adjust the index based on your layout preference


# (Plotting of wind barbs and other features remains the same)
# Add Radar, Geneva, and Lausanne locations to all plots
for row in axs:
    for ax in row:
        # Radar location or points of interest
        ax.plot(downburst_lon, downburst_lat, marker='x', color='red', markersize=16, 
            transform=ccrs.PlateCarree(), 
            path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
    
        # Set the spatial extent of the plot
        ax.set_extent([downburst_lon - 0.4, downburst_lon + 0.4, downburst_lat -0.2, downburst_lat + 0.2], 
                  crs=ccrs.PlateCarree())
    
        # Add geographic features
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=1)
        ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=1)
        ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=1)

from matplotlib.colors import to_rgba
lime_green_with_alpha = to_rgba("lime", alpha=0.5)

# Add TRT cells contour to every subplot
for row in axs:
    for ax in row:
        # Ensure cells, lons, and lats are aligned and properly scaled
        ax.contour(lon_trtc, lat_trtc, cells, levels=np.arange(0, 11), colors=[lime_green_with_alpha], alpha=0.5, transform=ccrs.PlateCarree())

# Iterate over rows in the filtered dataframe
for _, row in filtered_df.iterrows():
    try:
        # Extract values directly without wrapping them in float or np.array
        longitude = row['LONGITUDE']
        latitude = row['LATITUDE']
        speed = row['101']  # Wind speed
        direction = row['197']  # Wind direction

        # Ensure the values are 1-dimensional arrays
        longitude = np.array([longitude])  # Wrap as 1D array
        latitude = np.array([latitude])    # Wrap as 1D array
        speed = np.array([speed])
        direction = np.array([direction])
        
        # Convert to wind components
        u = -speed * np.sin(np.radians(direction)) * 3.6
        v = -speed * np.cos(np.radians(direction)) * 3.6
        
        for row in axs:
            for ax in row:
        # Add a single wind barb
                ax.barbs(longitude, latitude, u, v, transform=ccrs.PlateCarree(), color='black', length=8, linewidth=2.5, zorder=10)
                ax.barbs(longitude, latitude, u, v, transform=ccrs.PlateCarree(), color='white', length=8, linewidth=1, zorder=11)
    except ValueError as e:
        print(f"Skipping row due to error: {e}")



# Set main title
plt.suptitle(valid_time3 + ' UTC, ' + radar_name + ' elevation scan: ' + elevation_scan + ', Barbs: 10 min max s⁻¹ gust speed [km h⁻¹], Contour: TRT Cell', color=edgecolor, fontsize=16, x=0.5)
# Adjust layout and display the plot
plt.tight_layout(rect=[0, 0, 1, 0.95])

plt.savefig("/users/fackerma/newproject1/figures/second_case/TRT2/TRT_" + downburst_location + "_" + radar_station + "_" + valid_time3 + "_" + elevation_scan + "_2.png", dpi=300)

## RHI

In [ ]:
# Locarno 0700
downburst_location = 'Locarno'
downburst_x = 704167
downburst_y = 114317
downburst_z = 367

if radar_station == 'L':
    rhi_distance = 50
else:
    rhi_distance = 130

# Define time intervals: x minutes before, and the exact time
time_deltas = [-15, -10, -5, 0]  # minutes

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

radar_coords = (radar_lat, radar_lon)
downburst_coords = (downburst_lat, downburst_lon)
downburst_distance = geopy.distance.geodesic(radar_coords, downburst_coords).km

# Calculate azimuth
azimuth_to_downburst = calculate_azimuth(radar_lat, radar_lon, downburst_lat, downburst_lon)
print(f"Azimuth angle to downburst occurrence: {azimuth_to_downburst:.2f} degrees")

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

# Set up the figure with 4 rows and 2 columns
fig, axes = plt.subplots(len(time_deltas), 4, sharex=True, sharey=True, figsize=(30, 20))
fig.suptitle("2021-07-13 Corrected Velocity, Radial Shear, Reflectivity, Kdp. X = " + downburst_location + ", Radar: " + radar_name, fontsize=16)

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Proceed only if files are found
    if not files_rhi:
        print(f"No files found for time {current_time}. Skipping.")
        continue

    # Merge radar data for the current time
    for i, f in enumerate(files_rhi):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged = radar
        else:
            radar_merged = pyart.util.join_radar(radar_merged, radar)

    # Perform dealiasing
    corr_vel = pyart.correct.dealias_region_based(radar_merged)
    kdp = pyart.retrieve.kdp_vulpiani(radar_merged, psidp_field='uncorrected_differential_phase', band='C', parallel=True)

    #Add 'Kdp' field if it doesn't already exist
    # Add 'kdp' field if it doesn't already exist, wrapped in a dictionary with the 'data' key
    if 'kdp' not in radar_merged.fields:
        radar_merged.add_field('kdp', kdp[0])
    else:
        print(f"Field 'kdp' already exists for time {current_time}. Skipping addition.")
    
    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged.fields:
        radar_merged.add_field('corrected_velocity', corr_vel)
    else:
        print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")


    div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict = {
        'data': div_shear[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }

    # Now, add the field to the radar object
    radar_merged.add_field('div_shear', div_shear_dict)


    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi = pyart.util.cross_section_ppi(radar_merged, [azimuth_to_downburst])
    display = pyart.graph.RadarDisplay(pseudorhi)

    # Define axes for corrected velocity (left) and reflectivity (right)
    ax_vel = axes[row, 0]
    ax_ref = axes[row, 2]
    ax_kdp = axes[row, 3]
    ax_div_shear = axes[row, 1] 

    # Plot 'corrected_velocity' with specified color limits
    cmap_velocity.set_bad(color='grey')
    display.plot_rhi('corrected_velocity', ax=ax_vel, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
    ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for velocity
    #cbar = plt.colorbar(display.plots[0], ax=ax_vel)
    #cbar.set_label('[m s⁻¹]', fontsize=12)

    # Plot 'reflectivity'
    cmap_czc.set_bad(color='grey')
    display.plot_rhi('reflectivity', ax=ax_ref, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
    ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for reflectivity
    #cbar = plt.colorbar(display.plots[1], ax=ax_ref)
    #cbar.set_label('[dBZ]', fontsize=12)

    # Plot 'Kdp'
    cmap_Kdp.set_bad(color='grey')
    display.plot_rhi('kdp', ax=ax_kdp, ticks=Kdp_levels, norm=Kdp_norm, cmap=cmap_Kdp)
    ax_kdp.set_title(f"Kdp at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for Kdp
    #cbar = plt.colorbar(display.plots[2], ax=ax_kdp)
    #cbar.set_label('[° km⁻¹]', fontsize=12)

    # Plot 'div_shear'
    cmap_div_shear.set_bad(color='grey')
    display.plot_rhi('div_shear', ax=ax_div_shear, ticks=div_shear_levels, norm=div_shear_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_div_shear)
    ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for divergence shear
    #cbar = plt.colorbar(display.plots[3], ax=ax_div_shear)
    #cbar.set_label('[m s⁻¹ km⁻¹]', fontsize=12)

    # Mark the downburst distance on the x-axis with an 'X'
    for axis in [ax_vel, ax_ref, ax_kdp, ax_div_shear]:
        axis.annotate('X', xy=(downburst_distance, 0), xytext=(downburst_distance, -1),
                      textcoords='data', ha='center', color='red', fontsize=14,
                      arrowprops=dict(arrowstyle='-', color='red'))

    # Set common axis labels and limits
    ax_vel.set_ylim([0, 10])   # Adjust height range as needed
    ax_vel.set_xlim([0, rhi_distance])  # Adjust distance range as needed
    for axis in [ax_vel, ax_ref, ax_kdp, ax_div_shear
                 ]:
        axis.set_xlabel('Distance (km)')
        axis.set_ylabel('Height (km)')


plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
#plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station + "RHI_" + valid_time3 + "_" + downburst_location + "_2.png", dpi=300)

#plt.show()


In [ ]:
# hightemp Locaarno 0700
downburst_location = 'Locarno'
downburst_x = 704167
downburst_y = 114317
downburst_z = 367

if radar_station == 'L':
    rhi_distance = 50
else:
    rhi_distance = 130

# Define time intervals: x minutes before, and the exact time
time_deltas = [ -5, 0]  # minutes
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

radar_coords = (radar_lat, radar_lon)
downburst_coords = (downburst_lat, downburst_lon)
downburst_distance = geopy.distance.geodesic(radar_coords, downburst_coords).km

# Calculate azimuth
azimuth_to_downburst = calculate_azimuth(radar_lat, radar_lon, downburst_lat, downburst_lon)
print(f"Azimuth angle to downburst occurrence: {azimuth_to_downburst:.2f} degrees")

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(4, 3, sharex=True, sharey=True, figsize=(30, 20))
fig.suptitle("2021-07-13 Corrected Velocity, Radial Shear, Reflectivity, Kdp. X = " + downburst_location + ", Radar: " + radar_name, fontsize=16)

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")


    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)

    # Merge radar data for the current time
    for i, f in enumerate(files_rhi):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged = radar
        else:
            radar_merged = pyart.util.join_radar(radar_merged, radar)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged.fields:
        radar_merged.add_field('corrected_velocity', corr_vel)
    else:
        print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict = {
        'data': div_shear[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged.add_field('div_shear', div_shear_dict)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object
    
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi = pyart.util.cross_section_ppi(radar_merged, [azimuth_to_downburst])
    display = pyart.graph.RadarDisplay(pseudorhi)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst])
    display_first_scan = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst])
    display_second_scan = pyart.graph.RadarDisplay(pseudorhi_second_scan)

    if delta == -5:
        ax_vel_first_scan = axes[0, 0]
        ax_ref_first_scan = axes[0, 2]
        ax_div_shear_first_scan = axes[0, 1]
        ax_vel_second_scan = axes[1, 0]
        ax_ref_second_scan = axes[1, 2]
        ax_div_shear_second_scan = axes[1, 1]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC") 
    if delta == 0:
        ax_vel_first_scan = axes[2, 0]
        ax_ref_first_scan = axes[2, 2]
        ax_div_shear_first_scan = axes[2, 1]
        ax_vel_second_scan = axes[3, 0]
        ax_ref_second_scan = axes[3, 2]
        ax_div_shear_second_scan = axes[3, 1]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")
    
# Loop over rows
for row in range(4):  # Adjust depending on the number of rows in your layout
    for col in range(3):  # Adjust depending on the number of columns
        ax_vel = axes[row, 0]  # Velocity plot
        ax_ref = axes[row, 2]  # Reflectivity plot
        ax_div_shear = axes[row, 1]  # Divergence/Shear plot

        # Annotate 'X' on the axes where needed
        for axis in [ax_vel, ax_ref, ax_div_shear]:
            axis.annotate('X', xy=(downburst_distance, 0), xytext=(downburst_distance, -1),
                          textcoords='data', ha='center', color='red', fontsize=14,
                          arrowprops=dict(arrowstyle='-', color='red'))

        # Set axis limits for each axis
        ax_vel.set_ylim([0, 10])   # Adjust height range as needed
        ax_vel.set_xlim([0, rhi_distance])  # Adjust distance range as needed

        # Set labels for all axes
        ax_vel.set_xlabel('Distance (km)')
        ax_vel.set_ylabel('Height (km)')
        
        ax_ref.set_xlabel('Distance (km)')
        ax_ref.set_ylabel('Height (km)')

        ax_div_shear.set_xlabel('Distance (km)')
        ax_div_shear.set_ylabel('Height (km)')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station + "RHIhightemp_" + valid_time3 + "_" + downburst_location + "_2.png", dpi=300)



In [ ]:
# hightemp Locaarno 0700
downburst_location = 'Locarno'
downburst_x = 704167
downburst_y = 114317
downburst_z = 367

if radar_station == 'L':
    rhi_close = 0
    rhi_distance = 50
else:
    rhi_distance = 130
    rhi_close = 70

# Define time intervals: x minutes before, and the exact time
time_deltas = [-15, -10, -5, 0]  # minutes
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

radar_coords = (radar_lat, radar_lon)
downburst_coords = (downburst_lat, downburst_lon)
downburst_distance = geopy.distance.geodesic(radar_coords, downburst_coords).km

# Calculate azimuth
azimuth_to_downburst = calculate_azimuth(radar_lat, radar_lon, downburst_lat, downburst_lon)
print(f"Azimuth angle to downburst occurrence: {azimuth_to_downburst:.2f} degrees")

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

div_shear_levels = np.arange(-5,5.5,0.5)
div_shear_norm = BoundaryNorm(boundaries=div_shear_levels, ncolors=256, clip=True)

# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(8, 5, sharex=True, sharey=True, figsize=(30, 40))
fig.suptitle("2021-07-13 Corrected Velocity, Radial Shear, Reflectivity, Kdp, Zdr. X = " + downburst_location + ", Radar: " + radar_name, fontsize=16)

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)

    # Merge radar data for the current time
    for i, f in enumerate(files_rhi):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged = radar
        else:
            radar_merged = pyart.util.join_radar(radar_merged, radar)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    #corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    #if 'corrected_velocity' not in radar_merged.fields:
        #radar_merged.add_field('corrected_velocity', corr_vel)
    #else:
        #print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    #div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=1.5, r_k=2, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_first_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_first_scan.add_field('kdp', kdp[0])

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_second_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_second_scan.add_field('kdp', kdp[0])
 
    
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi = pyart.util.cross_section_ppi(radar_merged, [azimuth_to_downburst])
    display = pyart.graph.RadarDisplay(pseudorhi)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst])
    display_first_scan = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst])
    display_second_scan = pyart.graph.RadarDisplay(pseudorhi_second_scan)


    
        
    if delta == -15:
        ax_vel_first_scan = axes[0, 0]
        ax_ref_first_scan = axes[0, 2]
        ax_div_shear_first_scan = axes[0, 1]
        ax_kdp_first_scan = axes[0,3]
        ax_zdr_first_scan = axes[0,4]
        ax_vel_second_scan = axes[1, 0]
        ax_ref_second_scan = axes[1, 2]
        ax_div_shear_second_scan = axes[1, 1]
        ax_kdp_second_scan = axes[1,3]
        ax_zdr_second_scan = axes[1,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == -10:
        ax_vel_first_scan = axes[2, 0]
        ax_ref_first_scan = axes[2, 2]
        ax_div_shear_first_scan = axes[2, 1]
        ax_kdp_first_scan = axes[2,3]
        ax_zdr_first_scan = axes[2,4]
        ax_vel_second_scan = axes[3, 0]
        ax_ref_second_scan = axes[3, 2]
        ax_div_shear_second_scan = axes[3, 1]
        ax_kdp_second_scan = axes[3,3]
        ax_zdr_second_scan = axes[3,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == -5:
        ax_vel_first_scan = axes[4, 0]
        ax_ref_first_scan = axes[4, 2]
        ax_div_shear_first_scan = axes[4, 1]
        ax_kdp_first_scan = axes[4,3]
        ax_zdr_first_scan = axes[4,4]
        ax_vel_second_scan = axes[5, 0]
        ax_ref_second_scan = axes[5, 2]
        ax_div_shear_second_scan = axes[5, 1]
        ax_kdp_second_scan = axes[5,3]
        ax_zdr_second_scan = axes[5,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC") 

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == 0:
        ax_vel_first_scan = axes[6, 0]
        ax_ref_first_scan = axes[6, 2]
        ax_div_shear_first_scan = axes[6, 1]
        ax_kdp_first_scan = axes[6,3]
        ax_zdr_first_scan = axes[6,4]
        ax_vel_second_scan = axes[7, 0]
        ax_ref_second_scan = axes[7, 2]
        ax_div_shear_second_scan = axes[7, 1]
        ax_kdp_second_scan = axes[7,3]
        ax_zdr_second_scan = axes[7,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
    
# Loop over rows
for row in range(8):  # Adjust depending on the number of rows in your layout
    for col in range(5):  # Adjust depending on the number of columns
        ax_vel = axes[row, 0]  # Velocity plot
        ax_ref = axes[row, 2]  # Reflectivity plot
        ax_div_shear = axes[row, 1]  # Divergence/Shear plot
        ax_kdp = axes[row, 3]
        ax_zdr = axes[row, 4]

        # Annotate 'X' on the axes where needed
        for axis in [ax_vel, ax_ref, ax_div_shear, ax_kdp, ax_zdr]:
            axis.annotate('X', xy=(downburst_distance, 0), xytext=(downburst_distance, -1),
                          textcoords='data', ha='center', color='red', fontsize=14,
                          arrowprops=dict(arrowstyle='-', color='red'))

        # Set axis limits for each axis
        ax_vel.set_ylim([0, 10])   # Adjust height range as needed
        ax_vel.set_xlim([rhi_close, rhi_distance])  # Adjust distance range as needed

        # Set labels for all axes
        ax_vel.set_xlabel('Distance (km)')
        ax_vel.set_ylabel('Height (km)')
        
        ax_ref.set_xlabel('Distance (km)')
        ax_ref.set_ylabel('Height (km)')

        ax_div_shear.set_xlabel('Distance (km)')
        ax_div_shear.set_ylabel('Height (km)')

        ax_kdp.set_xlabel('Distance (km)')
        ax_kdp.set_ylabel('Height (km)')

        ax_zdr.set_xlabel('Distance (km)')
        ax_zdr.set_ylabel('Height (km)')



plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
plt.savefig("/users/fackerma/newproject1/figures/second_case/RHI3/" + radar_station + "RHIhightemp_" + valid_time3 + "_" + downburst_location + "_6.png", dpi=300)


In [ ]:
# Lugano 0650
downburst_location = 'Lugano'
downburst_x = 717874
downburst_y = 95884
downburst_z = 273

if radar_station == 'L':
    rhi_distance = 50
else:
    rhi_distance = 130

# Define time intervals: x minutes before, and the exact time
time_deltas = [-15, -10, -5, 0]  # minutes

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

radar_coords = (radar_lat, radar_lon)
downburst_coords = (downburst_lat, downburst_lon)
downburst_distance = geopy.distance.geodesic(radar_coords, downburst_coords).km

# Calculate azimuth
azimuth_to_downburst = calculate_azimuth(radar_lat, radar_lon, downburst_lat, downburst_lon)
print(f"Azimuth angle to downburst occurrence: {azimuth_to_downburst:.2f} degrees")

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

# Set up the figure with 4 rows and 2 columns
fig, axes = plt.subplots(len(time_deltas), 4, sharex=True, sharey=True, figsize=(30, 20))
fig.suptitle("2021-07-13 Corrected Velocity, Radial Shear, Reflectivity, Kdp. X = " + downburst_location + ", Radar: " + radar_name, fontsize=16)

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Proceed only if files are found
    if not files_rhi:
        print(f"No files found for time {current_time}. Skipping.")
        continue

    # Merge radar data for the current time
    for i, f in enumerate(files_rhi):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged = radar
        else:
            radar_merged = pyart.util.join_radar(radar_merged, radar)

    # Perform dealiasing
    corr_vel = pyart.correct.dealias_region_based(radar_merged)
    kdp = pyart.retrieve.kdp_vulpiani(radar_merged, psidp_field='uncorrected_differential_phase', band='C', parallel=True)

    #Add 'Kdp' field if it doesn't already exist
    # Add 'kdp' field if it doesn't already exist, wrapped in a dictionary with the 'data' key
    if 'kdp' not in radar_merged.fields:
        radar_merged.add_field('kdp', kdp[0])
    else:
        print(f"Field 'kdp' already exists for time {current_time}. Skipping addition.")
    
    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged.fields:
        radar_merged.add_field('corrected_velocity', corr_vel)
    else:
        print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")


    div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict = {
        'data': div_shear[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }

    # Now, add the field to the radar object
    radar_merged.add_field('div_shear', div_shear_dict)


    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi = pyart.util.cross_section_ppi(radar_merged, [azimuth_to_downburst])
    display = pyart.graph.RadarDisplay(pseudorhi)

    # Define axes for corrected velocity (left) and reflectivity (right)
    ax_vel = axes[row, 0]
    ax_ref = axes[row, 2]
    ax_kdp = axes[row, 3]
    ax_div_shear = axes[row, 1] 

    # Plot 'corrected_velocity' with specified color limits
    cmap_velocity.set_bad(color='grey')
    display.plot_rhi('corrected_velocity', ax=ax_vel, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
    ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for velocity
    #cbar = plt.colorbar(display.plots[0], ax=ax_vel)
    #cbar.set_label('[m s⁻¹]', fontsize=12)

    # Plot 'reflectivity'
    cmap_czc.set_bad(color='grey')
    display.plot_rhi('reflectivity', ax=ax_ref, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
    ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for reflectivity
    #cbar = plt.colorbar(display.plots[1], ax=ax_ref)
    #cbar.set_label('[dBZ]', fontsize=12)

    # Plot 'Kdp'
    cmap_Kdp.set_bad(color='grey')
    display.plot_rhi('kdp', ax=ax_kdp, ticks=Kdp_levels, norm=Kdp_norm, cmap=cmap_Kdp)
    ax_kdp.set_title(f"Kdp at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for Kdp
    #cbar = plt.colorbar(display.plots[2], ax=ax_kdp)
    #cbar.set_label('[° km⁻¹]', fontsize=12)

    # Plot 'div_shear'
    cmap_div_shear.set_bad(color='grey')
    display.plot_rhi('div_shear', ax=ax_div_shear, ticks=div_shear_levels, norm=div_shear_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_div_shear)
    ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")
    # Customize the colorbar for divergence shear
    #cbar = plt.colorbar(display.plots[3], ax=ax_div_shear)
    #cbar.set_label('[m s⁻¹ km⁻¹]', fontsize=12)

    # Mark the downburst distance on the x-axis with an 'X'
    for axis in [ax_vel, ax_ref, ax_kdp, ax_div_shear]:
        axis.annotate('X', xy=(downburst_distance, 0), xytext=(downburst_distance, -1),
                      textcoords='data', ha='center', color='red', fontsize=14,
                      arrowprops=dict(arrowstyle='-', color='red'))

    # Set common axis labels and limits
    ax_vel.set_ylim([0, 10])   # Adjust height range as needed
    ax_vel.set_xlim([0, rhi_distance])  # Adjust distance range as needed
    for axis in [ax_vel, ax_ref, ax_kdp, ax_div_shear
                 ]:
        axis.set_xlabel('Distance (km)')
        axis.set_ylabel('Height (km)')


plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
#plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station + "RHI_" + valid_time3 + "_" + downburst_location + "_2.png", dpi=300)

#plt.show()


In [ ]:
# hightemp Lugano 0650
downburst_location = 'Lugano'
downburst_x = 717874
downburst_y = 95884
downburst_z = 273

if radar_station == 'L':
    rhi_distance = 50
else:
    rhi_distance = 130

# Define time intervals: x minutes before, and the exact time
time_deltas = [ -5, 0]  # minutes
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

radar_coords = (radar_lat, radar_lon)
downburst_coords = (downburst_lat, downburst_lon)
downburst_distance = geopy.distance.geodesic(radar_coords, downburst_coords).km

# Calculate azimuth
azimuth_to_downburst = calculate_azimuth(radar_lat, radar_lon, downburst_lat, downburst_lon)
print(f"Azimuth angle to downburst occurrence: {azimuth_to_downburst:.2f} degrees")

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(4, 3, sharex=True, sharey=True, figsize=(30, 20))
fig.suptitle("2021-07-13 Corrected Velocity, Radial Shear, Reflectivity, Kdp. X = " + downburst_location + ", Radar: " + radar_name, fontsize=16)

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")


    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)

    # Merge radar data for the current time
    for i, f in enumerate(files_rhi):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged = radar
        else:
            radar_merged = pyart.util.join_radar(radar_merged, radar)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged.fields:
        radar_merged.add_field('corrected_velocity', corr_vel)
    else:
        print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict = {
        'data': div_shear[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged.add_field('div_shear', div_shear_dict)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object
    
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi = pyart.util.cross_section_ppi(radar_merged, [azimuth_to_downburst])
    display = pyart.graph.RadarDisplay(pseudorhi)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst])
    display_first_scan = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst])
    display_second_scan = pyart.graph.RadarDisplay(pseudorhi_second_scan)

    if delta == -5:
        ax_vel_first_scan = axes[0, 0]
        ax_ref_first_scan = axes[0, 2]
        ax_div_shear_first_scan = axes[0, 1]
        ax_vel_second_scan = axes[1, 0]
        ax_ref_second_scan = axes[1, 2]
        ax_div_shear_second_scan = axes[1, 1]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC") 
    if delta == 0:
        ax_vel_first_scan = axes[2, 0]
        ax_ref_first_scan = axes[2, 2]
        ax_div_shear_first_scan = axes[2, 1]
        ax_vel_second_scan = axes[3, 0]
        ax_ref_second_scan = axes[3, 2]
        ax_div_shear_second_scan = axes[3, 1]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")
    
# Loop over rows
for row in range(4):  # Adjust depending on the number of rows in your layout
    for col in range(3):  # Adjust depending on the number of columns
        ax_vel = axes[row, 0]  # Velocity plot
        ax_ref = axes[row, 2]  # Reflectivity plot
        ax_div_shear = axes[row, 1]  # Divergence/Shear plot

        # Annotate 'X' on the axes where needed
        for axis in [ax_vel, ax_ref, ax_div_shear]:
            axis.annotate('X', xy=(downburst_distance, 0), xytext=(downburst_distance, -1),
                          textcoords='data', ha='center', color='red', fontsize=14,
                          arrowprops=dict(arrowstyle='-', color='red'))

        # Set axis limits for each axis
        ax_vel.set_ylim([0, 10])   # Adjust height range as needed
        ax_vel.set_xlim([0, rhi_distance])  # Adjust distance range as needed

        # Set labels for all axes
        ax_vel.set_xlabel('Distance (km)')
        ax_vel.set_ylabel('Height (km)')
        
        ax_ref.set_xlabel('Distance (km)')
        ax_ref.set_ylabel('Height (km)')

        ax_div_shear.set_xlabel('Distance (km)')
        ax_div_shear.set_ylabel('Height (km)')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station + "RHIhightemp_" + valid_time3 + "_" + downburst_location + "_2.png", dpi=300)



In [ ]:
# hightemp Lugano 0650
downburst_location = 'Lugano'
downburst_x = 717874
downburst_y = 95884
downburst_z = 273

if radar_station == 'L':
    rhi_distance = 20
    rhi_close = 0
else:
    rhi_distance = 130
    rhi_close = 80

# Define time intervals: x minutes before, and the exact time
time_deltas = [-15, -10, -5, 0]  # minutes
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

radar_coords = (radar_lat, radar_lon)
downburst_coords = (downburst_lat, downburst_lon)
downburst_distance = geopy.distance.geodesic(radar_coords, downburst_coords).km

# Calculate azimuth
azimuth_to_downburst = calculate_azimuth(radar_lat, radar_lon, downburst_lat, downburst_lon)
print(f"Azimuth angle to downburst occurrence: {azimuth_to_downburst:.2f} degrees")

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

div_shear_levels = np.arange(-5,5.5,0.5)
div_shear_norm = BoundaryNorm(boundaries=div_shear_levels, ncolors=256, clip=True)

# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(8, 5, sharex=True, sharey=True, figsize=(30, 40))
fig.suptitle("2021-07-13 Corrected Velocity, Radial Shear, Reflectivity, Kdp, Zdr. X = " + downburst_location + ", Radar: " + radar_name, fontsize=16)

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)

    # Merge radar data for the current time
    for i, f in enumerate(files_rhi):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged = radar
        else:
            radar_merged = pyart.util.join_radar(radar_merged, radar)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    #corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    #if 'corrected_velocity' not in radar_merged.fields:
        #radar_merged.add_field('corrected_velocity', corr_vel)
    #else:
        #print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    #div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=1.5, r_k=2, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_first_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_first_scan.add_field('kdp', kdp[0])

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_second_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_second_scan.add_field('kdp', kdp[0])
 
    
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi = pyart.util.cross_section_ppi(radar_merged, [azimuth_to_downburst])
    display = pyart.graph.RadarDisplay(pseudorhi)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst])
    display_first_scan = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst])
    display_second_scan = pyart.graph.RadarDisplay(pseudorhi_second_scan)


    
        
    if delta == -15:
        ax_vel_first_scan = axes[0, 0]
        ax_ref_first_scan = axes[0, 2]
        ax_div_shear_first_scan = axes[0, 1]
        ax_kdp_first_scan = axes[0,3]
        ax_zdr_first_scan = axes[0,4]
        ax_vel_second_scan = axes[1, 0]
        ax_ref_second_scan = axes[1, 2]
        ax_div_shear_second_scan = axes[1, 1]
        ax_kdp_second_scan = axes[1,3]
        ax_zdr_second_scan = axes[1,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == -10:
        ax_vel_first_scan = axes[2, 0]
        ax_ref_first_scan = axes[2, 2]
        ax_div_shear_first_scan = axes[2, 1]
        ax_kdp_first_scan = axes[2,3]
        ax_zdr_first_scan = axes[2,4]
        ax_vel_second_scan = axes[3, 0]
        ax_ref_second_scan = axes[3, 2]
        ax_div_shear_second_scan = axes[3, 1]
        ax_kdp_second_scan = axes[3,3]
        ax_zdr_second_scan = axes[3,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == -5:
        ax_vel_first_scan = axes[4, 0]
        ax_ref_first_scan = axes[4, 2]
        ax_div_shear_first_scan = axes[4, 1]
        ax_kdp_first_scan = axes[4,3]
        ax_zdr_first_scan = axes[4,4]
        ax_vel_second_scan = axes[5, 0]
        ax_ref_second_scan = axes[5, 2]
        ax_div_shear_second_scan = axes[5, 1]
        ax_kdp_second_scan = axes[5,3]
        ax_zdr_second_scan = axes[5,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC") 

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == 0:
        ax_vel_first_scan = axes[6, 0]
        ax_ref_first_scan = axes[6, 2]
        ax_div_shear_first_scan = axes[6, 1]
        ax_kdp_first_scan = axes[6,3]
        ax_zdr_first_scan = axes[6,4]
        ax_vel_second_scan = axes[7, 0]
        ax_ref_second_scan = axes[7, 2]
        ax_div_shear_second_scan = axes[7, 1]
        ax_kdp_second_scan = axes[7,3]
        ax_zdr_second_scan = axes[7,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
    
# Loop over rows
for row in range(8):  # Adjust depending on the number of rows in your layout
    for col in range(5):  # Adjust depending on the number of columns
        ax_vel = axes[row, 0]  # Velocity plot
        ax_ref = axes[row, 2]  # Reflectivity plot
        ax_div_shear = axes[row, 1]  # Divergence/Shear plot
        ax_kdp = axes[row, 3]
        ax_zdr = axes[row, 4]

        # Annotate 'X' on the axes where needed
        for axis in [ax_vel, ax_ref, ax_div_shear, ax_kdp, ax_zdr]:
            axis.annotate('X', xy=(downburst_distance, 0), xytext=(downburst_distance, -1),
                          textcoords='data', ha='center', color='red', fontsize=14,
                          arrowprops=dict(arrowstyle='-', color='red'))

        # Set axis limits for each axis
        ax_vel.set_ylim([-0.5, 10])   # Adjust height range as needed
        ax_vel.set_xlim([rhi_close, rhi_distance])  # Adjust distance range as needed

        # Set labels for all axes
        ax_vel.set_xlabel('Distance (km)')
        ax_vel.set_ylabel('Height (km)')
        
        ax_ref.set_xlabel('Distance (km)')
        ax_ref.set_ylabel('Height (km)')

        ax_div_shear.set_xlabel('Distance (km)')
        ax_div_shear.set_ylabel('Height (km)')

        ax_kdp.set_xlabel('Distance (km)')
        ax_kdp.set_ylabel('Height (km)')

        ax_zdr.set_xlabel('Distance (km)')
        ax_zdr.set_ylabel('Height (km)')



plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
plt.savefig("/users/fackerma/newproject1/figures/second_case/RHI3/" + radar_station + "RHIhightemp_" + valid_time3 + "_" + downburst_location + "_6.png", dpi=300)


In [ ]:
# hightemp Lugano 0650
downburst_location = 'Matro'
downburst_x = 714263
downburst_y = 140944
downburst_z = 2171


if radar_station == 'L':
    rhi_distance = 20
    rhi_close = 0
else:
    rhi_distance = 100
    rhi_close = 50

# Define time intervals: x minutes before, and the exact time
time_deltas = [-15, -10, -5, 0]  # minutes
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

radar_coords = (radar_lat, radar_lon)
downburst_coords = (downburst_lat, downburst_lon)
downburst_distance = geopy.distance.geodesic(radar_coords, downburst_coords).km

# Calculate azimuth
azimuth_to_downburst = calculate_azimuth(radar_lat, radar_lon, downburst_lat, downburst_lon)
print(f"Azimuth angle to downburst occurrence: {azimuth_to_downburst:.2f} degrees")

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

div_shear_levels = np.arange(-5,5.5,0.5)
div_shear_norm = BoundaryNorm(boundaries=div_shear_levels, ncolors=256, clip=True)

# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(8, 5, sharex=True, sharey=True, figsize=(30, 40))
fig.suptitle("2021-07-13 Corrected Velocity, Radial Shear, Reflectivity, Kdp, Zdr. X = " + downburst_location + ", Radar: " + radar_name, fontsize=16)

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)

    # Merge radar data for the current time
    for i, f in enumerate(files_rhi):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged = radar
        else:
            radar_merged = pyart.util.join_radar(radar_merged, radar)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    #corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    #if 'corrected_velocity' not in radar_merged.fields:
        #radar_merged.add_field('corrected_velocity', corr_vel)
    #else:
        #print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    #div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=1.5, r_k=2, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_first_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_first_scan.add_field('kdp', kdp[0])

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_second_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_second_scan.add_field('kdp', kdp[0])
 
    
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi = pyart.util.cross_section_ppi(radar_merged, [azimuth_to_downburst])
    display = pyart.graph.RadarDisplay(pseudorhi)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst])
    display_first_scan = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst])
    display_second_scan = pyart.graph.RadarDisplay(pseudorhi_second_scan)


    
        
    if delta == -15:
        ax_vel_first_scan = axes[0, 0]
        ax_ref_first_scan = axes[0, 2]
        ax_div_shear_first_scan = axes[0, 1]
        ax_kdp_first_scan = axes[0,3]
        ax_zdr_first_scan = axes[0,4]
        ax_vel_second_scan = axes[1, 0]
        ax_ref_second_scan = axes[1, 2]
        ax_div_shear_second_scan = axes[1, 1]
        ax_kdp_second_scan = axes[1,3]
        ax_zdr_second_scan = axes[1,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == -10:
        ax_vel_first_scan = axes[2, 0]
        ax_ref_first_scan = axes[2, 2]
        ax_div_shear_first_scan = axes[2, 1]
        ax_kdp_first_scan = axes[2,3]
        ax_zdr_first_scan = axes[2,4]
        ax_vel_second_scan = axes[3, 0]
        ax_ref_second_scan = axes[3, 2]
        ax_div_shear_second_scan = axes[3, 1]
        ax_kdp_second_scan = axes[3,3]
        ax_zdr_second_scan = axes[3,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == -5:
        ax_vel_first_scan = axes[4, 0]
        ax_ref_first_scan = axes[4, 2]
        ax_div_shear_first_scan = axes[4, 1]
        ax_kdp_first_scan = axes[4,3]
        ax_zdr_first_scan = axes[4,4]
        ax_vel_second_scan = axes[5, 0]
        ax_ref_second_scan = axes[5, 2]
        ax_div_shear_second_scan = axes[5, 1]
        ax_kdp_second_scan = axes[5,3]
        ax_zdr_second_scan = axes[5,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC") 

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
    if delta == 0:
        ax_vel_first_scan = axes[6, 0]
        ax_ref_first_scan = axes[6, 2]
        ax_div_shear_first_scan = axes[6, 1]
        ax_kdp_first_scan = axes[6,3]
        ax_zdr_first_scan = axes[6,4]
        ax_vel_second_scan = axes[7, 0]
        ax_ref_second_scan = axes[7, 2]
        ax_div_shear_second_scan = axes[7, 1]
        ax_kdp_second_scan = axes[7,3]
        ax_zdr_second_scan = axes[7,4]

        # Plot 'corrected_velocity' with specified color limits
        cmap_velocity.set_bad(color='grey')
        display_first_scan.plot_rhi('corrected_velocity', ax=ax_vel_first_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_velocity.set_bad(color='grey')
        display_second_scan.plot_rhi('corrected_velocity', ax=ax_vel_second_scan, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity)
        #ax_vel.set_title(f"Corrected Velocity at {current_time[:2]}:{current_time[2:]} UTC")
 
        cmap_div_shear.set_bad(color='grey')
        display_first_scan.plot_rhi('div_shear', ax=ax_div_shear_first_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        cmap_div_shear.set_bad(color='grey')
        display_second_scan.plot_rhi('div_shear', ax=ax_div_shear_second_scan, ticks=div_shear_levels, norm=div_shear_norm, 
        #vmin=-20, vmax=70,
            cmap=cmap_div_shear)
        #ax_div_shear.set_title(f"Radial Shear at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_first_scan.plot_rhi('reflectivity', ax=ax_ref_first_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")  

        # Plot 'reflectivity'
        cmap_czc.set_bad(color='grey')
        display_second_scan.plot_rhi('reflectivity', ax=ax_ref_second_scan, ticks=czc_levels, norm=czc_norm, 
                     vmin=-20, vmax=70,
                     cmap=cmap_czc)
        #ax_ref.set_title(f"Reflectivity at {current_time[:2]}:{current_time[2:]} UTC")

        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_first_scan.plot_rhi('kdp', ax=ax_kdp_first_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot KDP
        cmap_Kdp.set_bad(color='grey')
        display_second_scan.plot_rhi('kdp', ax=ax_kdp_second_scan, ticks=Kdp_levels, norm=Kdp_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_Kdp)
        
        # Plot Zdr
        cmap_diff_reflectivity.set_bad(color='grey')
        display_first_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_first_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
        
        cmap_diff_reflectivity.set_bad(color='grey')
        display_second_scan.plot_rhi('differential_reflectivity', ax=ax_zdr_second_scan, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
                     #vmin=-20, vmax=70,
                     cmap=cmap_diff_reflectivity)
    
# Loop over rows
for row in range(8):  # Adjust depending on the number of rows in your layout
    for col in range(5):  # Adjust depending on the number of columns
        ax_vel = axes[row, 0]  # Velocity plot
        ax_ref = axes[row, 2]  # Reflectivity plot
        ax_div_shear = axes[row, 1]  # Divergence/Shear plot
        ax_kdp = axes[row, 3]
        ax_zdr = axes[row, 4]

        # Annotate 'X' on the axes where needed
        for axis in [ax_vel, ax_ref, ax_div_shear, ax_kdp, ax_zdr]:
            axis.annotate('X', xy=(downburst_distance, 0), xytext=(downburst_distance, -1),
                          textcoords='data', ha='center', color='red', fontsize=14,
                          arrowprops=dict(arrowstyle='-', color='red'))

        # Set axis limits for each axis
        ax_vel.set_ylim([-0.5, 10])   # Adjust height range as needed
        ax_vel.set_xlim([rhi_close, rhi_distance])  # Adjust distance range as needed

        # Set labels for all axes
        ax_vel.set_xlabel('Distance (km)')
        ax_vel.set_ylabel('Height (km)')
        
        ax_ref.set_xlabel('Distance (km)')
        ax_ref.set_ylabel('Height (km)')

        ax_div_shear.set_xlabel('Distance (km)')
        ax_div_shear.set_ylabel('Height (km)')

        ax_kdp.set_xlabel('Distance (km)')
        ax_kdp.set_ylabel('Height (km)')

        ax_zdr.set_xlabel('Distance (km)')
        ax_zdr.set_ylabel('Height (km)')



plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
plt.savefig("/users/fackerma/newproject1/figures/second_case/RHI3/" + radar_station + "RHIhightemp_" + valid_time3 + "_" + downburst_location + "_6.png", dpi=300)


## CAPPI

In [ ]:
# Assess CAPPI files
files_rhi = sorted(glob.glob(path + '/MLL/ML' + radar_station + '21194' + valid_time3 + '*')) # Read all elevations for rhi
for i,f in enumerate(files_rhi):
    radar = pyart.aux_io.read_metranet(f)
    
    if i == 0:
        radar_merged = radar
    else:
        radar_merged = pyart.util.join_radar(radar_merged, 
                                       radar)
corr_vel = pyart.correct.dealias_region_based(radar_merged)
radar_merged.add_field('corrected_velocity', corr_vel)


div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=2, r_k=1.5, resolution=0.5)

# Assuming div_shear is a NumPy array or similar
div_shear_dict = {
    'data': div_shear[1],  # This should be the numerical array
    'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
    'long_name': 'Divergence Shear',  # Add a descriptive name for the field
    'standard_name': 'div_shear',  # You can add any standard name
}

# Now, add the field to the radar object
radar_merged.add_field('div_shear', div_shear_dict)



In [ ]:
# Set definitions
zmin = 500
zmax = 8000
ymin= xmin = -100000
ymax = xmax = 100000
lat = float(radar.latitude['data'])
lon = float(radar.longitude['data'])
alt = float(radar.altitude['data'])
# number of grid points in cappi
cappi_res_h = 500
cappi_res_v = 500
ny = int((ymax-ymin)/cappi_res_h)+1
nx = int((xmax-xmin)/cappi_res_h)+1
nz = int((zmax-zmin)/cappi_res_v)+1

cappi_zh = pyart.map.grid_from_radars(radar_merged, grid_shape=(nz, ny, nx),
        grid_limits=((zmin, zmax), (ymin, ymax),
                     (xmin, xmax)),
        fields=['reflectivity', 'corrected_velocity', 'div_shear'])

display = pyart.graph.GridMapDisplay(cappi_zh)

In [ ]:
# Define the projection
projection = ccrs.PlateCarree()

# Downburst location (Swiss grid coordinates) - Locarno: 07:00
downburst_location = 'Locarno'
downburst_x = 704167
downburst_y = 114317
downburst_z = 367

# Convert Swiss coordinates to WGS84
wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

# Create the figure and subplots
fig = plt.figure(figsize=(18, 12))  # Increase figure height to accommodate three rows

# Define variable configurations: names, colormaps, levels, norms, and titles
variables = [
    ('corrected_velocity', cmap_velocity, velocity_levels, velocity_norm, 'Corrected Velocity [m s⁻¹]'),
    ('div_shear', cmap_div_shear, div_shear_levels, div_shear_norm, 'Radial Shear [m s⁻¹ km⁻¹]'),
    ('reflectivity', cmap_czc, czc_levels, czc_norm, 'Reflectivity [dBZ]')
]

# Define the levels for each row
levels = [0, 5, 10]

# Loop through rows and variables to create subplots
for row_idx, level in enumerate(levels):
    cappi_height = cappi_zh.z['data'][level] / 1000  # Convert height to km
    for col_idx, (var, cmap, levs, norm, title) in enumerate(variables):
        # Calculate subplot index
        subplot_idx = row_idx * len(variables) + col_idx + 1
        
        # Create subplot
        ax = plt.subplot(len(levels), len(variables), subplot_idx, projection=projection)
        
        # Plot the grid data
        display.plot_grid(
            var, level=level, ax=ax, projection=projection,
            cmap=cmap, norm=norm, vmin=levs[0], vmax=levs[-1],
            title=f"{title}\nHeight: {cappi_height:.1f} km" if row_idx == 0 else f"Height: {cappi_height:.1f} km"
        )
        
        # Highlight the downburst location
        ax.plot(downburst_lon, downburst_lat, marker='x', color='red', markersize=12, transform=ccrs.PlateCarree())
        
        # Set the map extent for all plots
        ax.set_extent([ downburst_lon-0.35 , downburst_lon+0.35, downburst_lat+0.25, downburst_lat-0.25], crs=ccrs.PlateCarree())
        
        # Add geographic features
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7)
        ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
        ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)
        
        # Remove axis labels and grid lines
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.grid(linestyle='-', color='black', alpha=0)




fig.suptitle("2021-07-13 " + valid_time3 +" UTC Velocity, Radial Shear, Reflectivity CAPPI, X = " + downburst_location +", Radar: " + radar_name, fontsize=16, ha='center')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
#plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station +"CAPPI_" + valid_time3 +"_" + downburst_location + "_1.png", dpi=300)





In [ ]:
# Define the projection
projection = ccrs.PlateCarree()

# Downburst location (Swiss grid coordinates) - Lugano: 06:50
downburst_location = 'Lugano'
downburst_x = 717874
downburst_y = 95884
downburst_z = 273

# Convert Swiss coordinates to WGS84
wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]

# Create the figure and subplots
fig = plt.figure(figsize=(18, 12))  # Increase figure height to accommodate three rows

# Define variable configurations: names, colormaps, levels, norms, and titles
variables = [
    ('corrected_velocity', cmap_velocity, velocity_levels, velocity_norm, 'Corrected Velocity [m s⁻¹]'),
    ('div_shear', cmap_div_shear, div_shear_levels, div_shear_norm, 'Radial Shear [m s⁻¹ km⁻¹]'),
    ('reflectivity', cmap_czc, czc_levels, czc_norm, 'Reflectivity [dBZ]')
]

# Define the levels for each row
levels = [0, 5, 10]

# Loop through rows and variables to create subplots
for row_idx, level in enumerate(levels):
    cappi_height = cappi_zh.z['data'][level] / 1000  # Convert height to km
    for col_idx, (var, cmap, levs, norm, title) in enumerate(variables):
        # Calculate subplot index
        subplot_idx = row_idx * len(variables) + col_idx + 1
        
        # Create subplot
        ax = plt.subplot(len(levels), len(variables), subplot_idx, projection=projection)
        
        # Plot the grid data
        display.plot_grid(
            var, level=level, ax=ax, projection=projection,
            cmap=cmap, norm=norm, vmin=levs[0], vmax=levs[-1],
            title=f"{title}\nHeight: {cappi_height:.1f} km" if row_idx == 0 else f"Height: {cappi_height:.1f} km"
        )
        
        # Highlight the downburst location
        ax.plot(downburst_lon, downburst_lat, marker='x', color='red', markersize=12, transform=ccrs.PlateCarree())
        
        # Set the map extent for all plots
        ax.set_extent([ downburst_lon-0.35 , downburst_lon+0.35, downburst_lat+0.25, downburst_lat-0.25], crs=ccrs.PlateCarree())
        
        # Add geographic features
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
        ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7)
        ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
        ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)
        
        # Remove axis labels and grid lines
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.grid(linestyle='-', color='black', alpha=0)




fig.suptitle("2021-07-13 " + valid_time3 +" UTC Velocity, Radial Shear, Reflectivity CAPPI, X = " + downburst_location + ", Radar: " + radar_name, fontsize=16, ha='center')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for the title
plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station +"CAPPI_" + valid_time3 +"_" + downburst_location + "_1.png", dpi=300)





## Optical Flow

In [ ]:
# Parameters for generating file paths
start_hour = int(valid_time3[:2])  # Extract the first two characters as the hour
start_time = int(valid_time3[2:])  # Extract the last two characters as the minutes
time_interval = 5  # Time interval between files in minutes
num_files = 10  

# Generate file paths dynamically based on the starting time and interval
file_paths = []
current_hour = start_hour
current_minute = start_time

for i in range(num_files):
    # Append the file path for the current time
    file_paths.append(
        f'/scratch/mch/fackerma/orders/case_130721/CZC/CZC21194{str(current_hour).zfill(2)}{str(current_minute).zfill(2)}VL.801'
    )
    # Update minutes and handle rollover to the next hour
    current_minute += time_interval
    if current_minute >= 60:
        current_minute -= 60
        current_hour += 1
        if current_hour >= 24:  # Handle 24-hour format rollover
            current_hour = 0

# Print file paths to verify
for path in file_paths:
    print(path)


# Initialize an empty list to hold the data arrays
czc_data_list = []

# Read the radar data and extract the data arrays
for file_path in file_paths:
    czc_data = radlib.read_file(file_path, physic_value=True)
    czc_data_list.append(czc_data.data)

# Stack the data into a 3D array with shape (t, x, y)
input_images = np.stack(czc_data_list, axis=0)
input_images = np.nan_to_num(input_images, nan=0.0, posinf=0.0, neginf=0.0)


# Compute motion field using Lucas-Kanade optical flow
#oflow = motion.lucaskanade.dense_lucaskanade(input_images)
oflow = motion.darts.DARTS(input_images)
#oflow = motion.proesmans.proesmans(input_images)

# Set the projection to EPSG:3035
projection = ccrs.epsg(3035)


# Extract u (horizontal) and v (vertical) motion components
u = oflow[0]
v = oflow[1]


# Create a figure with a single subplot
fig, ax = plt.subplots(figsize=(20, 12), subplot_kw={'projection': projection})

# Set background to black for the subplot
fig.patch.set_facecolor('white')  # Set figure background color
ax.set_facecolor('white')  # Set axis background color
edgecolor = 'black'

# Plot Reflectivity composite (czc)
cmap_czc = LinearSegmentedColormap.from_list("custom_cmap", ["white", "blue", "green", "yellow","red","magenta","black"], N=256)
#cmap_czc = plt.cm.inferno
cmap_czc.set_bad(color='grey')
p1 = ax.pcolormesh(clons, clats, czc, vmin=-10, vmax=60, 
                   cmap=cmap_czc, transform=projection)

# Add a colorbar
cbar1 = plt.colorbar(p1, ax=ax, extend='both', cmap=cmap_czc, norm=czc_norm, boundaries=czc_levels, ticks=czc_levels
                     )
cbar1.outline.set_edgecolor(edgecolor)
cbar1.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color=edgecolor, fontsize=20)

# Set title for the plot
ax.set_title('Z Composite [dBZ]', color=edgecolor, loc='right', fontsize=20)
ax.set_extent(TicinoLarge_extent, crs=ccrs.PlateCarree())

# Add borders, coastlines, and lakes with custom contour lines
ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=6)
ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7, linewidth=2)
ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)

clons_adj = clons[:-1, :-1]
clats_adj = clats[:-1, :-1]

step = 20  # Adjust to reduce density of barbs
clons_down = clons_adj[::step, ::step]
clats_down = clats_adj[::step, ::step]
u_down = u[::step, ::step]
v_down = v[::step, ::step]

u_down = u_down
v_down = -v_down

barbs = ax.quiver(
    clons_down, clats_down, u_down, v_down,
    transform=projection, color='black', alpha=1, edgecolor='black', linewidth=1, zorder=20,
    #length=7, linewidth=1, 
    regrid_shape=14  # Adjust length and regrid_shape
)

from matplotlib.colors import to_rgba
lime_green_with_alpha = to_rgba("lime", alpha=0.5)
ax.contour(lon_trtc, lat_trtc, cells, levels=np.arange(0, 11), #colors=[lime_green_with_alpha],
           colors='black',
            alpha=1, linestyles='dotted', transform=ccrs.PlateCarree())


#plt.savefig("/users/fackerma/newproject1/figures/second_case/OF/OF_" + valid_time3 + "_7" ".png", dpi=300)




## Custom Plots

In [ ]:
# Locarno RHIs for Weissfluhgipfel and Lema for 0656 UTC
downburst_location_locarno = 'Locarno'
downburst_x = 704167
downburst_y = 114317
downburst_z = 367

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat_locarno = wgs84[0]
downburst_lon_locarno = wgs84[1]

rhi_close_lema = 0
rhi_distance_lema = 40
rhi_close_weissfluhgipfel = 80
rhi_distance_weissfluhgipfel = 120


radar_lat_lema = 46.0401
radar_lon_lema = 8.8334
radar_alt_lema = 1627 


radar_lat_weissfluhgipfel = 46.832
radar_lon_weissfluhgipfel = 9.796
radar_alt_weissfluhgipfel = 2892 

radar_coords_lema = (radar_lat_lema, radar_lon_lema)
radar_coords_weissfluhgipfel = (radar_lat_weissfluhgipfel, radar_lon_weissfluhgipfel)
downburst_coords_locarno = (downburst_lat_locarno, downburst_lon_locarno)
downburst_distance_lema = geopy.distance.geodesic(radar_coords_lema, downburst_coords_locarno).km
downburst_distance_weissfluhgipfel = geopy.distance.geodesic(radar_coords_weissfluhgipfel, downburst_coords_locarno).km

# Calculate azimuth
azimuth_to_downburst_lema = calculate_azimuth(radar_lat_lema, radar_lon_lema, downburst_lat_locarno, downburst_lon_locarno)
azimuth_to_downburst_weissfluhgipfel = calculate_azimuth(radar_lat_weissfluhgipfel, radar_lon_weissfluhgipfel, downburst_lat_locarno, downburst_lon_locarno)
print(f"Azimuth angle to downburst occurrence Lema: {azimuth_to_downburst_lema:.2f} degrees")
print(f"Azimuth angle to downburst occurrence Weissfluhgipfel: {azimuth_to_downburst_weissfluhgipfel:.2f} degrees")







valid_time3 = '0700'
elevation_scan = '1'
radar_station = 'W'
radar_name = 'Weissfluhgipfel'

# Load radar data
path = '/scratch/mch/fackerma/orders/case_130721'
file = glob.glob(path + '/MLL/ML' + radar_station + '21194' + valid_time3 + '0U.00' + elevation_scan)
# Read the data for the plots
data = pyart.aux_io.read_metranet(file[0], reader='python')
velocity = data.get_field(0, 'velocity').data
reflectivity = data.get_field(0, 'reflectivity').data

# Define time intervals: x minutes before, and the exact time
time_deltas = [ 0]  # minute
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    #corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    #if 'corrected_velocity' not in radar_merged.fields:
        #radar_merged.add_field('corrected_velocity', corr_vel)
    #else:
        #print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    #div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=1.5, r_k=2, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_first_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_first_scan.add_field('kdp', kdp[0])

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_second_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_second_scan.add_field('kdp', kdp[0])
 

    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst_weissfluhgipfel])
    display_first_scan_weissfluhgipfel = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst_weissfluhgipfel])
    display_second_scan_weissfluhgipfel = pyart.graph.RadarDisplay(pseudorhi_second_scan)




# Defining valid time and elevation scan
valid_time3 = '0700'
elevation_scan = '1'
radar_station = 'L'
radar_name = 'Lema'

# Load radar data
path = '/scratch/mch/fackerma/orders/case_130721'
file = glob.glob(path + '/MLL/ML' + radar_station + '21194' + valid_time3 + '0U.00' + elevation_scan)
# Read the data for the plots
data = pyart.aux_io.read_metranet(file[0], reader='python')
velocity = data.get_field(0, 'velocity').data
reflectivity = data.get_field(0, 'reflectivity').data

# Define time intervals: x minutes before, and the exact time
time_deltas = [0]  # minute
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    #corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    #if 'corrected_velocity' not in radar_merged.fields:
        #radar_merged.add_field('corrected_velocity', corr_vel)
    #else:
        #print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    #div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=1.5, r_k=2, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_first_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_first_scan.add_field('kdp', kdp[0])

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_second_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_second_scan.add_field('kdp', kdp[0])
 
    
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst_lema])
    display_first_scan_lema = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst_lema])
    display_second_scan_lema = pyart.graph.RadarDisplay(pseudorhi_second_scan)

    
   



In [1]:


div_shear_norm = BoundaryNorm(boundaries=div_shear_levels, ncolors=256, clip=True)
# Define tick levels and normalization
div_shear_levels = np.arange(-5, 5.5, 1)  # Original tick locations

rhi_close_weissfluhgipfel = 81


# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(24, 12))


ax_vel_weissfluhgipfel = axes[0, 0]  # Velocity plot
ax_vel_lema = axes[0, 1]  # Velocity plot
ax_div_shear_weissfluhgipfel = axes[1, 0]  # Divergence/Shear plot
ax_div_shear_lema = axes[1, 1]  # Divergence/Shear plot
ax_vel_weissfluhgipfel.set_xlim([rhi_close_weissfluhgipfel, rhi_distance_weissfluhgipfel])
ax_vel_lema.set_xlim([rhi_close_lema, rhi_distance_lema])
ax_div_shear_weissfluhgipfel.set_xlim([rhi_close_weissfluhgipfel, rhi_distance_weissfluhgipfel])
ax_div_shear_lema.set_xlim([rhi_close_lema, rhi_distance_lema])



# Plot 'corrected_velocity' with specified color limits
cmap_velocity.set_bad(color='grey')
display_first_scan_weissfluhgipfel.plot_rhi('corrected_velocity', ax=ax_vel_weissfluhgipfel, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity, colorbar_label='', colorbar_flag=True)

cmap_div_shear.set_bad(color='grey')
display_first_scan_weissfluhgipfel.plot_rhi('div_shear', ax=ax_div_shear_weissfluhgipfel, ticks=div_shear_levels, norm=div_shear_norm, 
#vmin=-20, vmax=70,
    cmap=cmap_div_shear, colorbar_label='', colorbar_flag=True)

# Plot 'corrected_velocity' with specified color limits
cmap_velocity.set_bad(color='grey')
display_first_scan_lema.plot_rhi('corrected_velocity', ax=ax_vel_lema, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity, colorbar_label='', colorbar_flag=True)

cmap_div_shear.set_bad(color='grey')
display_first_scan_lema.plot_rhi('div_shear', ax=ax_div_shear_lema, ticks=div_shear_levels, norm=div_shear_norm, 
#vmin=-20, vmax=70,
    cmap=cmap_div_shear, colorbar_label='', colorbar_flag=True)
        


# Annotate 'X' on the axes where needed
for axis in [ax_vel_weissfluhgipfel, ax_div_shear_weissfluhgipfel]:
    axis.annotate('X', xy=(downburst_distance_weissfluhgipfel, 0), xytext=(downburst_distance_weissfluhgipfel, -0.5),
                          textcoords='data', ha='center', color='red', fontsize=22,
                          arrowprops=dict(arrowstyle='-', color='red'))
    
# Annotate 'X' on the axes where needed
for axis in [ax_vel_lema, ax_div_shear_lema]:
    axis.annotate('X', xy=(downburst_distance_lema, 0), xytext=(downburst_distance_lema, -0.5),
                          textcoords='data', ha='center', color='red', fontsize=22,
                          arrowprops=dict(arrowstyle='-', color='red'))

# Set axis limits for each axis
ax_vel_weissfluhgipfel.set_ylim([0, 10])   # Adjust height range as needed




# Set global font size for tick labels
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)



ax_vel_weissfluhgipfel.set_xlabel('')
ax_vel_weissfluhgipfel.set_ylabel('km above radar', fontsize=20)
ax_vel_weissfluhgipfel.set_title('')
ax_vel_weissfluhgipfel.set_title('m s⁻¹', fontsize=20, loc ='right')
ax_vel_weissfluhgipfel.set_title('A', fontsize=40, loc ='left')

ax_vel_lema.set_xlabel('')
ax_vel_lema.set_ylabel('')
ax_vel_lema.set_title('')
ax_vel_lema.set_title('m s⁻¹', fontsize=20, loc ='right')
ax_vel_lema.set_title('B', fontsize=40, loc ='left')

ax_div_shear_weissfluhgipfel.set_xlabel('')
ax_div_shear_weissfluhgipfel.set_ylabel('km above radar', fontsize=20)
ax_div_shear_weissfluhgipfel.set_title('')
ax_div_shear_weissfluhgipfel.set_title('m s⁻¹ km⁻¹', fontsize=20, loc ='right')
ax_div_shear_weissfluhgipfel.set_xlabel('km from radar', fontsize=20)

ax_div_shear_lema.set_xlabel('')
ax_div_shear_lema.set_ylabel('')
ax_div_shear_lema.set_title('')
ax_div_shear_lema.set_title('m s⁻¹ km⁻¹', fontsize=20, loc ='right')
ax_div_shear_lema.set_xlabel('km from radar', fontsize=20)

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)


plt.tight_layout()
#plt.savefig("/users/fackerma/newproject1/figures/second_case/double_RHI_Locarno_2.png", dpi=300)


NameError: name 'BoundaryNorm' is not defined

In [ ]:


div_shear_norm = BoundaryNorm(boundaries=div_shear_levels, ncolors=256, clip=True)
# Define tick levels and normalization
div_shear_levels = np.arange(-5, 5.5, 1)  # Original tick locations



# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(1, 3, sharex=False, sharey=True, figsize=(24, 6))



ax_ref_lema = axes[0]  # Velocity plot

ax_kdp_lema = axes[1]  # Divergence/Shear plot

ax_zdr_lema = axes[2]  # Divergence/Shear plot

ax_ref_lema.set_xlim([rhi_close_lema, rhi_distance_lema])

ax_kdp_lema.set_xlim([rhi_close_lema, rhi_distance_lema])

ax_zdr_lema.set_xlim([rhi_close_lema, rhi_distance_lema])


# Plot 'corrected_velocity' with specified color limits
cmap_czc.set_bad(color='grey')
display_second_scan_lema.plot_rhi('reflectivity', ax=ax_ref_lema, vmin=-30, vmax=30, ticks=czc_levels, norm=czc_norm, cmap=cmap_czc, colorbar_label='', colorbar_flag=True)

cmap_Kdp.set_bad(color='grey')
display_second_scan_lema.plot_rhi('kdp', ax=ax_kdp_lema, ticks=Kdp_levels, norm=Kdp_norm, 
#vmin=-20, vmax=70,
    cmap=cmap_Kdp, colorbar_label='', colorbar_flag=True)

cmap_diff_reflectivity.set_bad(color='grey')
display_second_scan_lema.plot_rhi('differential_reflectivity', ax=ax_zdr_lema, ticks=diff_reflectivity_levels, norm=diff_reflectivity_norm, 
#vmin=-20, vmax=70,
    cmap=cmap_diff_reflectivity, colorbar_label='', colorbar_flag=True)
        


# Annotate 'X' on the axes where needed

# Annotate 'X' on the axes where needed
for axis in [ax_ref_lema, ax_kdp_lema, ax_zdr_lema]:
    axis.annotate('X', xy=(downburst_distance_lema, 0), xytext=(downburst_distance_lema, -0.5),
                          textcoords='data', ha='center', color='red', fontsize=22,
                          arrowprops=dict(arrowstyle='-', color='red'))

# Set axis limits for each axis
ax_ref_lema.set_ylim([0, 10])   # Adjust height range as needed




# Set global font size for tick labels
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)




ax_ref_lema.set_xlabel('km from radar', fontsize =20)
ax_ref_lema.set_ylabel('km above radar', fontsize =20)
ax_ref_lema.set_title('')
ax_ref_lema.set_title('dBZ', fontsize=20, loc ='right')
#ax_ref_lema.set_title('B', fontsize=40, loc ='left')

ax_kdp_lema.set_xlabel('km from radar', fontsize =20)
ax_kdp_lema.set_ylabel('')
ax_kdp_lema.set_title('')
ax_kdp_lema.set_title('° km⁻¹', fontsize=20, loc ='right')

ax_zdr_lema.set_xlabel('km from radar', fontsize =20)
ax_zdr_lema.set_ylabel('')
ax_zdr_lema.set_title('')
ax_zdr_lema.set_title('dB', fontsize=20, loc ='right')


plt.tight_layout()
#plt.savefig("/users/fackerma/newproject1/figures/second_case/RHI_Locarno_0658_4.png", dpi=300)


In [ ]:
# Lugano RHIs for Weissfluhgipfel and Lema for 0656 UTC
downburst_location = 'Lugano'
downburst_x = 717874
downburst_y = 95884
downburst_z = 273

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat_locarno = wgs84[0]
downburst_lon_locarno = wgs84[1]

rhi_close_lema = 0
rhi_distance_lema = 40
rhi_close_weissfluhgipfel = 90
rhi_distance_weissfluhgipfel = 130


radar_lat_lema = 46.0401
radar_lon_lema = 8.8334
radar_alt_lema = 1627 


radar_lat_weissfluhgipfel = 46.832
radar_lon_weissfluhgipfel = 9.796
radar_alt_weissfluhgipfel = 2892 

radar_coords_lema = (radar_lat_lema, radar_lon_lema)
radar_coords_weissfluhgipfel = (radar_lat_weissfluhgipfel, radar_lon_weissfluhgipfel)
downburst_coords_locarno = (downburst_lat_locarno, downburst_lon_locarno)
downburst_distance_lema = geopy.distance.geodesic(radar_coords_lema, downburst_coords_locarno).km
downburst_distance_weissfluhgipfel = geopy.distance.geodesic(radar_coords_weissfluhgipfel, downburst_coords_locarno).km

# Calculate azimuth
azimuth_to_downburst_lema = calculate_azimuth(radar_lat_lema, radar_lon_lema, downburst_lat_locarno, downburst_lon_locarno)
azimuth_to_downburst_weissfluhgipfel = calculate_azimuth(radar_lat_weissfluhgipfel, radar_lon_weissfluhgipfel, downburst_lat_locarno, downburst_lon_locarno)
print(f"Azimuth angle to downburst occurrence Lema: {azimuth_to_downburst_lema:.2f} degrees")
print(f"Azimuth angle to downburst occurrence Weissfluhgipfel: {azimuth_to_downburst_weissfluhgipfel:.2f} degrees")







valid_time3 = '0645'
elevation_scan = '1'
radar_station = 'W'
radar_name = 'Weissfluhgipfel'

# Load radar data
path = '/scratch/mch/fackerma/orders/case_130721'
file = glob.glob(path + '/MLL/ML' + radar_station + '21194' + valid_time3 + '0U.00' + elevation_scan)
# Read the data for the plots
data = pyart.aux_io.read_metranet(file[0], reader='python')
velocity = data.get_field(0, 'velocity').data
reflectivity = data.get_field(0, 'reflectivity').data

# Define time intervals: x minutes before, and the exact time
time_deltas = [ 0]  # minute
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    #corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    #if 'corrected_velocity' not in radar_merged.fields:
        #radar_merged.add_field('corrected_velocity', corr_vel)
    #else:
        #print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    #div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=1.5, r_k=2, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_first_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_first_scan.add_field('kdp', kdp[0])

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_second_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_second_scan.add_field('kdp', kdp[0])
 

    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst_weissfluhgipfel])
    display_first_scan_weissfluhgipfel = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst_weissfluhgipfel])
    display_second_scan_weissfluhgipfel = pyart.graph.RadarDisplay(pseudorhi_second_scan)




# Defining valid time and elevation scan
valid_time3 = '0645'
elevation_scan = '1'
radar_station = 'L'
radar_name = 'Lema'

# Load radar data
path = '/scratch/mch/fackerma/orders/case_130721'
file = glob.glob(path + '/MLL/ML' + radar_station + '21194' + valid_time3 + '0U.00' + elevation_scan)
# Read the data for the plots
data = pyart.aux_io.read_metranet(file[0], reader='python')
velocity = data.get_field(0, 'velocity').data
reflectivity = data.get_field(0, 'reflectivity').data

# Define time intervals: x minutes before, and the exact time
time_deltas = [0]  # minute
first_scan = ['001', '003', '005', '007', '009', '011', '013', '015', '017', '019']
second_scan = ['002', '004', '006', '008', '010', '012', '014', '016', '018', '020']

# Convert valid_time3 to a datetime object
base_time = datetime.strptime(valid_time3, "%H%M")

for row, delta in enumerate(time_deltas):
    # Calculate the time for this row
    current_time = (base_time + timedelta(minutes=delta)).strftime("%H%M")
    print(f"Looking for files with time: {current_time}")
    
    # Check what files are available in the directory
    files_rhi = sorted(glob.glob(f"{path}/MLL/ML{radar_station}21194{current_time}*"))
    print(f"Files found for time {current_time}: {files_rhi}")

    # Separate the files into two lists
    file_rhi_first_scan = []
    file_rhi_second_scan = []

    for file in files_rhi:
        # Extract the elevation scan number (last two digits before the file extension)
        elevation_scan = file.split('.')[-1]
    
        if elevation_scan in first_scan:
            file_rhi_first_scan.append(file)
        elif elevation_scan in second_scan:
            file_rhi_second_scan.append(file)
    
    # Merge radar data for first scan
    radar_merged_first_scan = None
    for i, f in enumerate(file_rhi_first_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_first_scan = radar
        else:
            radar_merged_first_scan = pyart.util.join_radar(radar_merged_first_scan, radar)

    # Merge radar data for second scan  
    radar_merged_second_scan = None
    for i, f in enumerate(file_rhi_second_scan):
        radar = pyart.aux_io.read_metranet(f)
        if i == 0:
            radar_merged_second_scan = radar
        else:
            radar_merged_second_scan = pyart.util.join_radar(radar_merged_second_scan, radar)

    # Perform dealiasing
    #corr_vel = pyart.correct.dealias_region_based(radar_merged)
    corr_vel_first_scan = pyart.correct.dealias_region_based(radar_merged_first_scan)
    corr_vel_second_scan = pyart.correct.dealias_region_based(radar_merged_second_scan)

    # Add 'corrected_velocity' field if it doesn't already exist
    #if 'corrected_velocity' not in radar_merged.fields:
        #radar_merged.add_field('corrected_velocity', corr_vel)
    #else:
        #print(f"Field 'corrected_velocity' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_first_scan.fields:
        radar_merged_first_scan.add_field('corrected_velocity', corr_vel_first_scan)
    else:
        print(f"Field 'corrected_velocity_first_scan' already exists for time {current_time}. Skipping addition.")

    # Add 'corrected_velocity' field if it doesn't already exist
    if 'corrected_velocity' not in radar_merged_second_scan.fields:
        radar_merged_second_scan.add_field('corrected_velocity', corr_vel_second_scan)
    else:
        print(f"Field 'corrected_velocity_second_scan' already exists for time {current_time}. Skipping addition.")

    #div_shear = llsd(vel=radar_merged.fields['corrected_velocity']['data'], az_min=3, az_max=90, w_k=1.5, r_k=2, resolution=0.5)
    div_shear_first_scan = llsd(vel=radar_merged_first_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)
    div_shear_second_scan = llsd(vel=radar_merged_second_scan.fields['corrected_velocity']['data'], az_min=3, az_max=10, w_k=5, r_k=5, resolution=0.5)

    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_first_scan = {
        'data': div_shear_first_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_first_scan.add_field('div_shear', div_shear_dict_first_scan)
    # Assuming div_shear is a NumPy array or similar
    div_shear_dict_second_scan = {
        'data': div_shear_second_scan[1],  # This should be the numerical array
        'units': 'm/s/km',  # You can adjust the units based on the actual meaning of div_shear
        'long_name': 'Divergence Shear',  # Add a descriptive name for the field
        'standard_name': 'div_shear',  # You can add any standard name
    }
    radar_merged_second_scan.add_field('div_shear', div_shear_dict_second_scan)
    # Now, add the field to the radar object

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_first_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_first_scan.add_field('kdp', kdp[0])

    kdp = pyart.retrieve.kdp_vulpiani(radar_merged_second_scan, psidp_field='uncorrected_differential_phase', band='C', parallel=True)
    radar_merged_second_scan.add_field('kdp', kdp[0])
 
    
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_first_scan = pyart.util.cross_section_ppi(radar_merged_first_scan, [azimuth_to_downburst_lema])
    display_first_scan_lema = pyart.graph.RadarDisplay(pseudorhi_first_scan)
    # Create a pseudo RHI cross-section at azimuth 270 degrees
    pseudorhi_second_scan = pyart.util.cross_section_ppi(radar_merged_second_scan, [azimuth_to_downburst_lema])
    display_second_scan_lema = pyart.graph.RadarDisplay(pseudorhi_second_scan)

    
   



In [ ]:


div_shear_norm = BoundaryNorm(boundaries=div_shear_levels, ncolors=256, clip=True)
# Define tick levels and normalization
div_shear_levels = np.arange(-5, 5.5, 1)  # Original tick locations

rhi_close_lema = 0
rhi_distance_lema = 20
rhi_close_weissfluhgipfel = 101
rhi_distance_weissfluhgipfel = 130

# Set up the figure with 4 rows and 4 columns
fig, axes = plt.subplots(2, 2, sharex=False, sharey=True, figsize=(24, 12))


ax_vel_weissfluhgipfel = axes[0, 0]  # Velocity plot
ax_vel_lema = axes[0, 1]  # Velocity plot
ax_div_shear_weissfluhgipfel = axes[1, 0]  # Divergence/Shear plot
ax_div_shear_lema = axes[1, 1]  # Divergence/Shear plot
ax_vel_weissfluhgipfel.set_xlim([rhi_close_weissfluhgipfel, rhi_distance_weissfluhgipfel])
ax_vel_lema.set_xlim([rhi_close_lema, rhi_distance_lema])
ax_div_shear_weissfluhgipfel.set_xlim([rhi_close_weissfluhgipfel, rhi_distance_weissfluhgipfel])
ax_div_shear_lema.set_xlim([rhi_close_lema, rhi_distance_lema])



# Plot 'corrected_velocity' with specified color limits
cmap_velocity.set_bad(color='grey')
display_first_scan_weissfluhgipfel.plot_rhi('corrected_velocity', ax=ax_vel_weissfluhgipfel, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity, colorbar_label='', colorbar_flag=True)

cmap_div_shear.set_bad(color='grey')
display_first_scan_weissfluhgipfel.plot_rhi('div_shear', ax=ax_div_shear_weissfluhgipfel, ticks=div_shear_levels, norm=div_shear_norm, 
#vmin=-20, vmax=70,
    cmap=cmap_div_shear, colorbar_label='', colorbar_flag=True)

# Plot 'corrected_velocity' with specified color limits
cmap_velocity.set_bad(color='grey')
display_first_scan_lema.plot_rhi('corrected_velocity', ax=ax_vel_lema, vmin=-30, vmax=30, ticks=velocity_levels, norm=velocity_norm, cmap=cmap_velocity, colorbar_label='', colorbar_flag=True)

cmap_div_shear.set_bad(color='grey')
display_first_scan_lema.plot_rhi('div_shear', ax=ax_div_shear_lema, ticks=div_shear_levels, norm=div_shear_norm, 
#vmin=-20, vmax=70,
    cmap=cmap_div_shear, colorbar_label='', colorbar_flag=True)
        


# Annotate 'X' on the axes where needed
for axis in [ax_vel_weissfluhgipfel, ax_div_shear_weissfluhgipfel]:
    axis.annotate('X', xy=(downburst_distance_weissfluhgipfel, 0), xytext=(downburst_distance_weissfluhgipfel, -0.5),
                          textcoords='data', ha='center', color='red', fontsize=22,
                          arrowprops=dict(arrowstyle='-', color='red'))
    
# Annotate 'X' on the axes where needed
for axis in [ax_vel_lema, ax_div_shear_lema]:
    axis.annotate('X', xy=(downburst_distance_lema, 0), xytext=(downburst_distance_lema, -0.5),
                          textcoords='data', ha='center', color='red', fontsize=22,
                          arrowprops=dict(arrowstyle='-', color='red'))

# Set axis limits for each axis
ax_vel_weissfluhgipfel.set_ylim([0, 10])   # Adjust height range as needed




# Set global font size for tick labels
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)



ax_vel_weissfluhgipfel.set_xlabel('')
ax_vel_weissfluhgipfel.set_ylabel('km above radar', fontsize=20)
ax_vel_weissfluhgipfel.set_title('')
ax_vel_weissfluhgipfel.set_title('m s⁻¹', fontsize=20, loc ='right')
ax_vel_weissfluhgipfel.set_title('A', fontsize=40, loc ='left')

ax_vel_lema.set_xlabel('')
ax_vel_lema.set_ylabel('')
ax_vel_lema.set_title('')
ax_vel_lema.set_title('m s⁻¹', fontsize=20, loc ='right')
ax_vel_lema.set_title('B', fontsize=40, loc ='left')

ax_div_shear_weissfluhgipfel.set_xlabel('')
ax_div_shear_weissfluhgipfel.set_ylabel('km above radar', fontsize=20)
ax_div_shear_weissfluhgipfel.set_title('')
ax_div_shear_weissfluhgipfel.set_title('m s⁻¹ km⁻¹', fontsize=20, loc ='right')
ax_div_shear_weissfluhgipfel.set_xlabel('km from radar', fontsize=20)

ax_div_shear_lema.set_xlabel('')
ax_div_shear_lema.set_ylabel('')
ax_div_shear_lema.set_title('')
ax_div_shear_lema.set_title('m s⁻¹ km⁻¹', fontsize=20, loc ='right')
ax_div_shear_lema.set_xlabel('km from radar', fontsize=20)


plt.tight_layout()
#plt.savefig("/users/fackerma/newproject1/figures/second_case/double_RHI_Lugano_0643_2.png", dpi=300)


In [ ]:
# Optical flow and Kdp of Locarno at 0650
projection = ccrs.epsg(3035)
fig, axs = plt.subplots(1, 2, figsize=(24, 8), subplot_kw={'projection': projection})
# Set the projection to EPSG:3035

# Set background to black for the subplot
fig.patch.set_facecolor('white')  # Set figure background color
#ax.set_facecolor('white')  # Set axis background color
edgecolor = 'black'
cmap_czc = LinearSegmentedColormap.from_list("custom_cmap", ["white", "blue", "green", "yellow","red","magenta","black"], N=256)


valid_time3 = '0650'

downburst_location = 'Locarno'
downburst_x = 704167
downburst_y = 114317
downburst_z = 367

wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
downburst_lat = wgs84[0]
downburst_lon = wgs84[1]


# Load radar data
path = '/scratch/mch/fackerma/orders/case_130721'
file8 = glob.glob(path +'/CZC/CZC21194' + valid_time3 + 'VL.801') # Reflectivity composite
czc_data = radlib.read_file(file8[0],physic_value=True)
czc=czc_data.data

path_trtc = {"lomdata": "/scratch/mch/fackerma/orders/case_130721/"}  # Replace with your actual path
trt_df, cells_list, timelist = read_TRT(path_trtc, ttime=valid_time3)
# Extract the first element from the list of cells
cells = cells_list[0]
# Define Swiss CH1903 to WGS84 transformer
transformer_trtc = Transformer.from_crs("epsg:21781", "epsg:4326", always_xy=True)
# Grid dimensions and origin
o_x, o_y = 255000, -160000  # Origin in Swiss coordinates
lx, ly = 710, 640           # Grid size (width x height)
cell_size = 1000            # 1 km grid spacing
# Create Swiss grid (CH1903 coordinates)
x_coords = np.linspace(o_x, o_x + (lx - 1) * cell_size, lx)
y_coords = np.linspace(o_y, o_y + (ly - 1) * cell_size, ly)
x_grid, y_grid = np.meshgrid(x_coords, y_coords)
# Convert grid to WGS84 (longitude, latitude)
lon_trtc, lat_trtc = transformer_trtc.transform(x_grid, y_grid)

# Parameters for generating file paths
start_hour = int(valid_time3[:2])  # Extract the first two characters as the hour
start_time = int(valid_time3[2:])  # Extract the last two characters as the minutes
time_interval = 5  # Time interval between files in minutes
num_files = 10  
ax= axs[0]

# Generate file paths dynamically based on the starting time and interval
file_paths = []
current_hour = start_hour
current_minute = start_time

for i in range(num_files):
    # Append the file path for the current time
    file_paths.append(
        f'/scratch/mch/fackerma/orders/case_130721/CZC/CZC21194{str(current_hour).zfill(2)}{str(current_minute).zfill(2)}VL.801'
    )
    # Update minutes and handle rollover to the next hour
    current_minute += time_interval
    if current_minute >= 60:
        current_minute -= 60
        current_hour += 1
        if current_hour >= 24:  # Handle 24-hour format rollover
            current_hour = 0

# Print file paths to verify
for path in file_paths:
    print(path)

# Initialize an empty list to hold the data arrays
czc_data_list = []

# Read the radar data and extract the data arrays
for file_path in file_paths:
    czc_data = radlib.read_file(file_path, physic_value=True)
    czc_data_list.append(czc_data.data)

# Stack the data into a 3D array with shape (t, x, y)
input_images = np.stack(czc_data_list, axis=0)
input_images = np.nan_to_num(input_images, nan=0.0, posinf=0.0, neginf=0.0)

oflow = motion.darts.DARTS(input_images)

u = oflow[0]
v = oflow[1]

ax=axs[0]
cmap_czc.set_bad(color='grey')
p1 = ax.pcolormesh(clons, clats, czc, vmin=-10, vmax=60, 
                   cmap=cmap_czc, transform=projection)

# Add a colorbar
cbar1 = plt.colorbar(p1, ax=ax, extend='both', cmap=cmap_czc, norm=czc_norm, boundaries=czc_levels, ticks=czc_levels
                     )
cbar1.outline.set_edgecolor(edgecolor)
cbar1.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color=edgecolor, fontsize=20)
ticks = cbar1.get_ticks()  # Get all ticks
cbar1.set_ticks(ticks[::2])
# Set title for the plot
ax.set_title('dBZ', color=edgecolor, loc='right', fontsize=20)
ax.set_extent([downburst_lon - 1, downburst_lon + 1, downburst_lat -0.5, downburst_lat + 0.5], crs=ccrs.PlateCarree())

# Add borders, coastlines, and lakes with custom contour lines
ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=6)
ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7, linewidth=2)
ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)

clons_adj = clons[:-1, :-1]
clats_adj = clats[:-1, :-1]

step = 20  # Adjust to reduce density of barbs
clons_down = clons_adj[::step, ::step]
clats_down = clats_adj[::step, ::step]
u_down = u[::step, ::step]
v_down = v[::step, ::step]

u_down = u_down
v_down = -v_down

barbs = ax.quiver(
    clons_down, clats_down, u_down, v_down,
    transform=projection, color='black', alpha=1, edgecolor='black', linewidth=1, zorder=20,
    #length=7, linewidth=1, 
    regrid_shape=14  # Adjust length and regrid_shape
)

ax.contour(lon_trtc, lat_trtc, cells, levels=np.arange(0, 11), #colors=[lime_green_with_alpha],
           colors='black',
            alpha=1, transform=ccrs.PlateCarree())






valid_time3 = '0650'
radar_station = 'W'
radar_name = 'Weissfluhgipfel'
elevation_scan = '3'
if radar_station=='W':
    radar_lat = 46.832
    radar_lon = 9.796
    radar_alt = 2892 

# Cimetta
downburst_x = 704434
downburst_y = 117458
downburst_z = 1661
wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
cimetta_lat = wgs84[0]
cimetta_lon = wgs84[1]

# Lugano
downburst_x = 717874
downburst_y = 95884
downburst_z = 273
wgs84 = converter.LV03toWGS84(downburst_x, downburst_y, downburst_z)
print(wgs84)
lugano_lat = wgs84[0]
lugano_lon = wgs84[1]

# Magadino airport
magadino_lat = 46.163159
magadino_lon = 8.877448

# Load radar data
path = '/scratch/mch/fackerma/orders/case_130721'
file8 = glob.glob(path +'/CZC/CZC21194' + valid_time3 + 'VL.801') # Reflectivity composite

file = glob.glob(path + '/MLL/ML' + radar_station + '21194' + valid_time3 + '0U.00' + elevation_scan)
file2 = glob.glob(path +'/DVL/srn/data/tmp/mof/DV' + radar_station + '/DV' + radar_station +'21194' + valid_time3 + '7L.80' + elevation_scan) # Dealiased velocity


# Read the data for the plots
data = pyart.aux_io.read_metranet(file[0], reader='python')
velocity = data.get_field(0, 'velocity').data
reflectivity = data.get_field(0, 'reflectivity').data

dv_data = pyart.aux_io.read_file_py(file2[0], physic_value=False)
dv_digital = dv_data.data
dv_header = dv_data.header
nyq = float(dv_header['nyquist'])
dv_velocity = transform_from_digital(dv_digital, nyq)

resolution = 0.5
range_max = reflectivity.shape[1] * resolution
azimuths = np.linspace(0, 360, reflectivity.shape[0])
ranges = np.arange(0, range_max, resolution)



# Transform radar polar coordinates to lat/lon
lats, lons = radar_to_latlon(radar_lat, radar_lon, ranges, azimuths)

path_trtc = {"lomdata": "/scratch/mch/fackerma/orders/case_130721/"}  # Replace with your actual path
trt_df, cells_list, timelist = read_TRT(path_trtc, ttime=valid_time3)
# Extract the first element from the list of cells
cells = cells_list[0]
# Define Swiss CH1903 to WGS84 transformer
transformer_trtc = Transformer.from_crs("epsg:21781", "epsg:4326", always_xy=True)
# Grid dimensions and origin
o_x, o_y = 255000, -160000  # Origin in Swiss coordinates
lx, ly = 710, 640           # Grid size (width x height)
cell_size = 1000            # 1 km grid spacing
# Create Swiss grid (CH1903 coordinates)
x_coords = np.linspace(o_x, o_x + (lx - 1) * cell_size, lx)
y_coords = np.linspace(o_y, o_y + (ly - 1) * cell_size, ly)
x_grid, y_grid = np.meshgrid(x_coords, y_coords)
# Convert grid to WGS84 (longitude, latitude)
lon_trtc, lat_trtc = transformer_trtc.transform(x_grid, y_grid)


ax=axs[1]
cmap_velocity.set_bad(color='grey')
p1 = ax.pcolormesh(lons, lats, dv_velocity, #vmin=-10, vmax=60, 
                   cmap=cmap_velocity, transform=ccrs.PlateCarree(), norm=velocity_norm)

# Add a colorbar
cbar1 = plt.colorbar(p1, ax=ax, extend='both', cmap=cmap_velocity, norm=velocity_norm, boundaries=velocity_levels, ticks=velocity_levels
                     )
cbar1.outline.set_edgecolor(edgecolor)
cbar1.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar1.ax.axes, 'yticklabels'), color=edgecolor, fontsize=20)
ticks = cbar1.get_ticks()  # Get all ticks
cbar1.set_ticks(ticks[::2])

# Set title for the plot
ax.set_title('m s⁻¹', color=edgecolor, loc='right', fontsize=20)
ax.set_extent([downburst_lon - 0.5, downburst_lon + 0.7, downburst_lat -0.3, downburst_lat + 0.3], crs=ccrs.PlateCarree())
ax.plot(cimetta_lon, cimetta_lat, marker='x', color='yellow', markersize=22, transform=ccrs.PlateCarree(), path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
ax.plot(lugano_lon, lugano_lat, marker='x', color='yellow', markersize=22, transform=ccrs.PlateCarree(), path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
ax.plot(magadino_lon, magadino_lat, marker='x', color='yellow', markersize=22, transform=ccrs.PlateCarree(), path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])
# Add borders, coastlines, and lakes with custom contour lines
ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=6)
ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7, linewidth=2)
ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)



ax.contour(lon_trtc, lat_trtc, cells, levels=np.arange(0, 11), #colors=[lime_green_with_alpha],
           colors='black',
            alpha=1, transform=ccrs.PlateCarree())







from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

# Define the size of the size bar (in map units, e.g., meters)
size_bar_length = 20000  # 10 km in meters (adjust as needed)
size_bar_text = '20 km'  # Text to display next to the size bar

# Create the size bar font
size_bar_font = fm.FontProperties(size=20, weight='bold')

# Add size bar to the second plot
size_bar_ax2 = AnchoredSizeBar(
    axs[1].transData,  # Transformation for positioning
    size_bar_length,    # Length of the size bar
    size_bar_text,      # Label for the size bar
    loc='lower left',   # Location on the plot
    pad=0.5,            # Padding around the bar
    color='black',      # Color of the size bar
    frameon=False,      # No frame around the bar
    size_vertical=50,   # Thickness of the bar
    fontproperties=size_bar_font  # Font properties for the label
)
axs[1].add_artist(size_bar_ax2)  # Add the size bar to the second axis

plt.tight_layout()

#plt.savefig("/users/fackerma/newproject1/figures/second_case/OF_Vel_1.png", dpi=300)




## Crowd Sourcing Data

In [ ]:
# Crowd-sourcing data and Max. downdraft estimator w
# Multi-variable plot
projection = ccrs.epsg(3035)

fig, ax = plt.subplots(figsize=(24, 12), subplot_kw={'projection': projection})


fig.patch.set_facecolor('white')  # Set figure background
ax.set_facecolor('white')  # Set axis background
edgecolor = 'black'

wge_levels = np.arange(2,20,2)
wge_norm = BoundaryNorm(boundaries=wge_levels, ncolors=256, clip=True)
cmap_wge = LinearSegmentedColormap.from_list("custom_cmap", ["white", "yellow", "orange", "red", "black"], N=256)
# Plot wge composite
# Set a mask
wge = np.ma.masked_less_equal(wge, 0)
cmap_wge.set_bad(color='grey')
p12 = ax.pcolormesh(clons, clats, wge, #vmin=0, vmax=18, 
                   cmap=cmap_wge, transform=projection, norm=wge_norm)
# Add a colorbar
cbar12 = plt.colorbar(p12, ax=ax, extend='both', cmap=cmap_wge, norm=wge_norm, boundaries=wge_levels, ticks=wge_levels
                     )
cbar12.outline.set_edgecolor(edgecolor)
cbar12.ax.yaxis.set_tick_params(color=edgecolor)
plt.setp(plt.getp(cbar12.ax.axes, 'yticklabels'), color=edgecolor)
ax.set_title('Max. Downdraft Velocity w [m s⁻¹]', color=edgecolor, loc='right')


            #ax.plot(radar_lon, radar_lat, marker='x', color='yellow', markersize=12, transform=ccrs.PlateCarree())  # Radar location
ax.plot(locarno_lon, locarno_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())  # Geneva
ax.plot(lugano_lon, lugano_lat, marker='o', color='white', markersize=8, markeredgecolor='black', markeredgewidth=2, transform=ccrs.PlateCarree())  # Lausanne
ax.set_extent([7.6, 9.75 , 45.35, 46.8], crs=ccrs.PlateCarree())
 # Add borders, coastlines, and lakes with custom contour lines
ax.add_feature(cfeature.BORDERS, edgecolor='black', linestyle='-', alpha=0.8, linewidth=6)
ax.add_feature(cfeature.COASTLINE, edgecolor='black', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='none', alpha=0.8, linewidth=2)
ax.add_feature(cfeature.BORDERS, edgecolor='white', linestyle='-', alpha=0.7)
ax.add_feature(cfeature.COASTLINE, edgecolor='white', alpha=0.7)
ax.add_feature(cfeature.LAKES, edgecolor='white', facecolor='none', alpha=0.7)
# Add city names with a black outline for visibility
locarno_text = ax.text(locarno_lon + 0.04, locarno_lat, 'Locarno', color='white', fontsize=14, transform=ccrs.PlateCarree())
lugano_text = ax.text(lugano_lon + 0.04, lugano_lat, 'Lugano', color='white', fontsize=14, transform=ccrs.PlateCarree())
locarno_text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])
lugano_text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])


ax.contour(lon_trtc, lat_trtc, cells, levels=np.arange(0, 11), colors='black', alpha=1, transform=ccrs.PlateCarree())



# Save the figure
#plt.savefig("/users/fackerma/newproject1/figures/second_case/" + radar_station + valid_time3 + "_" + elevation_scan + "_all_variables.png", facecolor='black', dpi=300)

In [14]:
from collections import Counter
# Filter for WIND reports and collect 'auspraegung' values
wind_auspragungen = [feature['properties']['auspraegung'] for feature in features 
                     if feature['properties']['category'] == 'WIND']

# Count occurrences of each 'auspraegung'
auspraegung_counts = Counter(wind_auspragungen)

print("WIND 'auspraegung' values and their counts:")
for auspraegung, count in auspraegung_counts.items():
    print(f"{auspraegung}: {count}")

print(f"\nTotal number of WIND reports: {len(wind_auspragungen)}")
print(f"Number of unique 'auspraegung' values: {len(auspraegung_counts)}")

WIND 'auspraegung' values and their counts:
WIND_STURM: 90
WIND_SCHWACH: 1613
WIND_STARK: 2235
WIND_STUERMISCH: 846
WIND_ORKAN: 67

Total number of WIND reports: 4851
Number of unique 'auspraegung' values: 5


In [16]:
import json
from datetime import datetime

file_path = '/scratch/mch/fackerma/orders/CS/met_crowd_meldungen_2021.json'

def convert_timestamp(ms):
    return datetime.utcfromtimestamp(ms / 1000).strftime('%Y-%m-%d %H:%M:%S')

with open(file_path, 'r') as file:
    data = json.load(file)


features = data['features']  # Remove the [0] indexing

# Get the first and last timestamps
timestamps = [f['properties']['timestamp'] for f in features]
first_timestamp = min(timestamps)
last_timestamp = max(timestamps)

print(f"First timestamp: {first_timestamp} -> {convert_timestamp(first_timestamp)}")
print(f"Last timestamp: {last_timestamp} -> {convert_timestamp(last_timestamp)}")

# Print a few examples
print("\nSome example conversions:")
for feature in features[:5]:  # First 5 features
    ts = feature['properties']['timestamp']
    submission_ts = feature['properties']['submissionTimestamp']
    print(f"Timestamp: {ts} -> {convert_timestamp(ts)}")
    print(f"Submission Timestamp: {submission_ts} -> {convert_timestamp(submission_ts)}")
    print("---")

First timestamp: 1633104000000 -> 2021-10-01 16:00:00
Last timestamp: 1640990858386 -> 2021-12-31 22:47:38

Some example conversions:
Timestamp: 1633104000000 -> 2021-10-01 16:00:00
Submission Timestamp: 1636452970232 -> 2021-11-09 10:16:10
---
Timestamp: 1633156200000 -> 2021-10-02 06:30:00
Submission Timestamp: 1634103304042 -> 2021-10-13 05:35:04
---
Timestamp: 1633172744945 -> 2021-10-02 11:05:44
Submission Timestamp: 1634110145847 -> 2021-10-13 07:29:05
---
Timestamp: 1633450147000 -> 2021-10-05 16:09:07
Submission Timestamp: 1636305164858 -> 2021-11-07 17:12:44
---
Timestamp: 1633586724245 -> 2021-10-07 06:05:24
Submission Timestamp: 1633586728643 -> 2021-10-07 06:05:28
---


/tmp/ipykernel_74661/3329205960.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(ms / 1000).strftime('%Y-%m-%d %H:%M:%S')


In [5]:
import json
from datetime import datetime
from collections import defaultdict

file_path = '/scratch/mch/fackerma/orders/CS/met_crowd_meldungen_2022.json'

def convert_timestamp(ms):
    return datetime.utcfromtimestamp(ms / 1000)

with open(file_path, 'r') as file:
    data = json.load(file)

features = data['features']

# Create a dictionary to store the earliest timestamp for each month
monthly_earliest = defaultdict(lambda: float('inf'))

# Iterate through all features
for feature in features:
    ts = feature['properties']['timestamp']
    date = convert_timestamp(ts)
    month_key = (date.year, date.month)
    
    # Update the earliest timestamp for this month
    monthly_earliest[month_key] = min(monthly_earliest[month_key], ts)

# Print the earliest timestamp for each month
print("Earliest timestamp for each month in 2021:")
for (year, month), timestamp in sorted(monthly_earliest.items()):
    if year == 2022:
        date_str = convert_timestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{year}-{month:02d}: {date_str}")

# Count reports per month
monthly_counts = defaultdict(int)
for feature in features:
    ts = feature['properties']['timestamp']
    date = convert_timestamp(ts)
    month_key = (date.year, date.month)
    monthly_counts[month_key] += 1

print("\nNumber of reports per month in 2021:")
for (year, month), count in sorted(monthly_counts.items()):
    if year == 2022:
        print(f"{year}-{month:02d}: {count}")


/tmp/ipykernel_16304/2975479707.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(ms / 1000)


Earliest timestamp for each month in 2021:
2022-01: 2022-01-01 00:07:34
2022-02: 2022-02-01 00:01:13
2022-03: 2022-03-01 00:10:17
2022-04: 2022-04-01 00:00:09
2022-05: 2022-05-01 01:37:50
2022-06: 2022-06-01 00:28:22
2022-07: 2022-07-01 00:03:57
2022-08: 2022-08-01 01:30:14
2022-09: 2022-09-01 01:41:26
2022-10: 2022-10-01 01:39:25
2022-11: 2022-11-01 01:36:57
2022-12: 2022-12-01 00:32:50

Number of reports per month in 2021:
2021-12: 20


In [9]:
import json
from datetime import datetime

file_path = '/scratch/mch/fackerma/orders/CS/met_crowd_meldungen_2023.json'

def convert_timestamp(ms):
    return datetime.utcfromtimestamp(ms / 1000)

# Define the target date and wind conditions
target_date = datetime(2023, 7, 24).date()
wind_conditions = ['WIND_STURM', 'WIND_STARK', 'WIND_STUERMISCH', 'WIND_ORKAN']

# Load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

features = data['features']

# Filter the features
filtered_features = []
for feature in features:
    props = feature['properties']
    date = convert_timestamp(props['timestamp']).date()
    if date == target_date and props['category'] == 'WIND' and props['auspraegung'] in wind_conditions:
        filtered_features.append(feature)

# Print results
print(f"Number of matching reports on {target_date}: {len(filtered_features)}")

# Print details of each matching report
for feature in filtered_features:
    props = feature['properties']
    timestamp = convert_timestamp(props['timestamp'])
    print(f"\nTime: {timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Location: {props['place']}")
    print(f"Wind condition: {props['auspraegung']}")
    print(f"Coordinates: {feature['geometry']['coordinates']}")

# Optionally, save the filtered features to a new JSON file
output_file = 'wind_reports_2023_07_24.json'
with open(output_file, 'w') as f:
    json.dump(filtered_features, f, indent=2)
print(f"\nFiltered reports saved to {output_file}")

/tmp/ipykernel_16304/2162057357.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(ms / 1000)


Number of matching reports on 2023-07-24: 604

Time: 2023-07-24 00:19:32
Location: Plan-les-Ouates
Wind condition: WIND_ORKAN
Coordinates: [6.1186699, 46.1686912]

Time: 2023-07-24 00:22:14
Location: Genève
Wind condition: WIND_STUERMISCH
Coordinates: [6.146138040354611, 46.19279016744949]

Time: 2023-07-24 00:23:58
Location: Petit-Lancy
Wind condition: WIND_STARK
Coordinates: [6.12, 46.185]

Time: 2023-07-24 00:26:24
Location: Genève
Wind condition: WIND_STARK
Coordinates: [6.1320601, 46.2125611]

Time: 2023-07-24 00:29:18
Location: Meyrin
Wind condition: WIND_STARK
Coordinates: [6.0715667, 46.2323354]

Time: 2023-07-24 00:30:00
Location: Rueyres-Treyfayes
Wind condition: WIND_STUERMISCH
Coordinates: [6.95958258, 46.64953615]

Time: 2023-07-24 00:30:10
Location: Genève
Wind condition: WIND_STURM
Coordinates: [6.152200421282323, 46.19037722935276]

Time: 2023-07-24 00:30:38
Location: Vésenaz
Wind condition: WIND_STARK
Coordinates: [6.200151378716307, 46.238155848731054]

Time: 2023-07-